In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from datetime import datetime
from tqdm import tqdm
warnings.filterwarnings(action = 'ignore')

# 2017, 2018 데이터 모델링

기본 전처리는 null값 중간값, 평균으로 채우기 및 기계 오류 행 제거순으로 진행되었고, 뒤에서 보여드릴 2019,2020과 동일하여 따로 언급하지 않겠습니다.

다만 2017, 2018에 관해서만 진행한 전처리가 있어 코드에 포함하도록 하겠습니다.

train, test data의 object type data word 각각을 sentence로 변경하는 함수 입니다. 미리 완성된 데이터가 있으므로, 시간 관계상 함수만 포함하도록 합니다.

In [7]:
def make_word_to_string(train_df, test_df):
    "function of making word to sentence"
    "input df must be make up with word(string type) dataframe"
    "if item is '단어1', '단어2', '단어3' --> '단어1 단어2 단어3'"
    datas = []
    nlp = [train_df,test_df]
    for nlp_data in nlp:
        input_ids = []
        for idx in range(len(nlp_data)):
            origin = list(nlp_data.iloc[idx])
            sent = ''
            for token in origin[:-1]:
                if (token != '없음') & (token != '모름') & (token != '없다'):
                    try:
                        sent += ' ' + token.replace(',','')
                    except:
                        continue
            input_ids.append(sent[1:])
        new = pd.DataFrame()
        new['input_ids'] = input_ids
        datas.append(new)
    return datas

In [8]:
from konlpy.tag import Okt
from konlpy.tag import Kkma

okt = Okt()
kkma = Kkma()

2017, 2018 train data를 이용해 미리 만들어 놓은 word count data를 불러옵니다.

In [9]:
key_word_2017 = pd.read_csv('word_count_0.csv');key_word_2017
key_word_2018 = pd.read_csv('word_count_1.csv');key_word_2018

,word,count
0,컴퓨터,3105
1,교육,2861
2,기사,2589
3,프로그램,2423
4,증가,2194
...,...,...
1365,주입,21
1366,실제,21
1367,주사,21
1368,법인,21


해당 data들을 이용하여 word list를 만들어 봅니다.

In [10]:
word_2017 = list(key_word_2017.word)
word_2018 = list(key_word_2018.word)

train, test data의 단어 요소 갯수를 세어 one-hot-vector를 가진 feature로 변환하는 함수 입니다.

작은 tip이지만 아래 함수를 이용하면 정말 빠르게 one-hot-vetor를 생성할 수 있습니다! 다중 for문을 돌릴때는 작업하는 col과 row의 방향이 일치하도록 코드를 작성하는 것이 알고리즘적으로 좋습니다 :)

In [11]:
def make_word_to_vector(datas, raw_datas, word_list):
    "function of word to one-hot-vector"
    "first input is list of sentence dataframe and third input is word list"
    "second input is list of raw dataframe"
    for n in range(len(2)):
        plus_col = []
        for w in range(len(word_list)):
            plus_col.append([0 for i in range(len(datas[n]))])
        for idx in tqdm(range(len(datas[n]))):
            temp_list = okt.nouns(datas[n]['input_ids'].iloc[idx])
            for i in range(len(word_list)):
                if word_list[i] in temp_list:
                    plus_col[i][idx] = 1
        for i in range(len(plus_col)):
            raw_datas[n][word_list[i]] = plus_col[i]
    return raw_datas

저희는 단어 one-hot-vector를 만드는 것에 그치지 않고, 이를 이용하여 새로운 feature를 도출할 방안에 대하여 고민하였습니다. train data에는 직업코드가 존재하고, 그에 따른 단어들의 one-hot-vector가 존재한다는 것을 고려하여 직업 코드 별 단어들의 누적 갯수와 data 각각의 유클리디안 거리를 계산하여 feature로 사용하였습니다.

### Distance feature를 위한 세가지 단계
1. 누적 one-hot-vecotr를 knowcode별 scaling 해준다. -> 직업코드별 갯수가 다르므로 규모에 영향을 받을 수 있다.
2. raw data의 각각 raw의 단어 vector와 유클리디안 distance를 구한다.
3. 유클리디안 distance가 가장 작은 knowcode 집합에 대하여 가중치를 주기 위한 feature를 생성한다.

In [12]:
def make_distance_feature(token_data, make_data, y):
    "function of make distance feature"
    "token_data는 word feature로 이루어진 train data"
    "make_data는 feature 생성할 train or test data"
    label_token = {}
    for line in tqdm(range(len(token_data))):
        try:
            label_token[y[line]] = [x+y for x, y in zip(label_token[y[line]], list(token_data.iloc[line]))]
        except:
            label_token[y[line]] = list(token_data.iloc[line])

    df = pd.DataFrame(label_token)
    df.index = token_data.columns
    df
    def minmax_norm(df_input):
        return (df - df.min()) / ( df.max() - df.min())
    new_df = minmax_norm(df)
    new_df = new_df.transpose() 
    labels = list(new_df.index)
    result = []
    for idx in tqdm(range(len(make_data))):
        a = np.array(make_data.iloc[idx])
        temp = []
        if sum(a) == 0:
            result.append(0)
            continue
        for tdx in range(len(new_df)):
            b = np.array(new_df.iloc[tdx])
            dist = np.linalg.norm(a-b)
            temp.append(dist)
        result.append(labels[temp.index(min(temp))])
    return result

위 함수로 생성된 feature를 2017, 2018 각각에 추가하여 새로운 data를 만들었습니다.

In [3]:
train_2017 = pd.read_csv('train_2017_dist.csv',index_col=[0])
test_2017 = pd.read_csv('test_2017_dist.csv',index_col=[0])
train_2018 = pd.read_csv('train_2018_dist.csv',index_col=[0])
test_2018 = pd.read_csv('test_2018_dist.csv',index_col=[0])

In [4]:
y_2017 = pd.read_csv('wc_2017_train.csv').knowcode
y_2018 = pd.read_csv('/home/chaeyoon-jang/test_env/train_2018_Non.csv')
y_2018 = y_2018.knowcode

앞서 새로 만든 'distance' feature를 포함한 몇몇 feature들에 대하여 categorical 처리를 해주어야 합니다.

In [14]:
train_2017['bq1'] = pd.Categorical(train_2017.bq1)
train_2017['bq2'] = pd.Categorical(train_2017.bq2)
train_2017['bq39_2'] = pd.Categorical(train_2017.bq39_2)
train_2017['dist'] = pd.Categorical(train_2017.dist)

test_2017['bq1'] = pd.Categorical(test_2017.bq1)
test_2017['bq2'] = pd.Categorical(test_2017.bq2)
test_2017['bq39_2'] = pd.Categorical(test_2017.bq39_2)
test_2017['dist'] = pd.Categorical(test_2017.dist)

In [5]:
train_2018= train_2018.astype(int)
test_2018= test_2018.astype(int)

train_2018['bq1'] = pd.Categorical(train_2018.bq1)
train_2018['bq2'] = pd.Categorical(train_2018.bq2)
train_2018['bq13'] = pd.Categorical(train_2018.bq13)
train_2018['bq15'] = pd.Categorical(train_2018.bq15)
train_2018['bq17'] = pd.Categorical(train_2018.bq17)
train_2018['bq18'] = pd.Categorical(train_2018.bq18)
train_2018['bq19'] = pd.Categorical(train_2018.bq19)
train_2018['dist'] = pd.Categorical(train_2018.dist)

test_2018['bq1'] = pd.Categorical(test_2018.bq1)
test_2018['bq2'] = pd.Categorical(test_2018.bq2)
test_2018['bq13'] = pd.Categorical(test_2018.bq13)
test_2018['bq15'] = pd.Categorical(test_2018.bq15)
test_2018['bq17'] = pd.Categorical(test_2018.bq17)
test_2018['bq18'] = pd.Categorical(test_2018.bq18)
test_2018['bq19'] = pd.Categorical(test_2018.bq19)
test_2018['dist'] = pd.Categorical(test_2018.dist)

In [16]:
for col in train_2017:
        train_2017[col].replace(' ', 0, inplace = True)
        if col != 'knowcode':
            test_2017[col].replace(' ', 0, inplace = True)
for col in train_2017:
        train_2017[col].replace(np.nan, 0, inplace = True)
        if col != 'knowcode':
            test_2017[col].replace(np.nan, 0, inplace = True)

# 모델

저희는 세가지를 고려하여 모델을 랜덤포레스트로 결정하였습니다.

1. train, test 데이터의 갯수가 비슷하거나, test가 더 많다.(오류행 제거 등의 이유로)
2. feature의 갯수가 무수히 많으며, 각각을 scaling하는 작업에는 무리가 있다.
3. 일반적인 설문지를 고려해보았을 때, 나뭇가지 처럼 나아가 어느 한 점으로 귀결되는 특성이 있다. (성격 심리 유형 검사 등) 

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# 2017 data train 결과

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(train_2017, y_2017, test_size=0.1, stratify=y_2017, random_state=777)

In [21]:
md = RandomForestClassifier(n_estimators=2000, random_state=777)
md.fit(X_train, y_train)

RandomForestClassifier(n_estimators=2000, random_state=777)

In [22]:
y_train_hat = md.predict(X_train)
print(accuracy_score(y_train,y_train_hat))
y_valid_hat = md.predict(X_valid)
print(accuracy_score(y_valid, y_valid_hat))
print(f1_score(y_train,y_train_hat, average='macro'))
print(f1_score(y_valid, y_valid_hat, average='macro'))

1.0
0.743139407244786
1.0
0.7000367946981112


In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(train_2018, y_2018, test_size=0.1, stratify=y_2018, random_state=777)

In [7]:
md = RandomForestClassifier(n_estimators=2000, random_state=777)
md.fit(X_train, y_train)

RandomForestClassifier(n_estimators=2000, random_state=777)

In [8]:
y_train_hat = md.predict(X_train)
print(accuracy_score(y_train,y_train_hat))
y_valid_hat = md.predict(X_valid)
print(accuracy_score(y_valid, y_valid_hat))
print(f1_score(y_train,y_train_hat, average='macro'))
print(f1_score(y_valid, y_valid_hat, average='macro'))

1.0
0.7234513274336283
1.0
0.6750629741918941


# 2019, 2020 데이터 모델링
## 앞선 두 데이터와 코드 작성자가 달라 파일 확장자 및 경로가 일치하지 않음을 미리 말씀드립니다.

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

### 연도별 데이터를 불러온다.

연도별 데이터는 각기 다른 특징 값들을 갖고 있기 때문에 각기 다른 변수에 할당해 줘야 합니다. 데이터에 대한 자세한 사항은 'KNOW 메타데이터' 폴더를 통해서 확인할 수 있습니다.

In [ ]:
train_2017 = []
train_2018 = []
train_2019 = []
train_2020 = []

for i, path in enumerate(sorted(glob("/content/gdrive/MyDrive/KNOW_data/train/*.csv"))):
    if i == 0:
        train_2017 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/train/KNOW_2017.csv')
        pass
    elif i==1:
        train_2018 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/train/KNOW_2018.csv')
        pass
    elif i==2:
        train_2019 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/train/KNOW_2019.csv')
        pass
    else:
        train_2020 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/train/KNOW_2020.csv')
        pass
    
test_2017 = []
test_2018 = []
test_2019 = []
test_2020 = []

for i, path in enumerate(sorted(glob("/content/gdrive/MyDrive/KNOW_data/test/*.csv"))):
    if i == 0:
        test_2017 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/test/KNOW_2017_test.csv')
        pass
    elif i==1:
        test_2018 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/test/KNOW_2018_test.csv')
        pass
    elif i==2:
        test_2019 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/test/KNOW_2019_test.csv')
        pass
    else:
        test_2020 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/test/KNOW_2020_test.csv')
        pass

### 2019 데이터를 확인해 보도록 하겠습니다.

### 1.1 공백으로 구성된 결측치 값을 np.nan으로 변경

데이터에 포함된 모든 결측치 값들은 스페이스를 통해서 나타나고 있습니다. 따라서, 이러한 결측치 값을 확인하기 편하도록 모두 np.nan 값으로 변형해줍니다.

In [ ]:
# 데이터의 결측치 값이 ' '기준으로 나타나기 때문에 결측치가 보이지 않는다.
train_2019.isnull().sum()

idx         0
sq1         0
sq2         0
sq3         0
sq4         0
           ..
bq30        3
bq31_1      3
bq31_2      3
bq31_3      9
knowcode    0
Length: 153, dtype: int64

In [ ]:
train_2019.columns

Index(['idx', 'sq1', 'sq2', 'sq3', 'sq4', 'sq5', 'sq6', 'sq7', 'sq8', 'sq9',
       ...
       'bq27_1', 'bq28', 'bq28_1', 'bq28_2', 'bq29', 'bq30', 'bq31_1',
       'bq31_2', 'bq31_3', 'knowcode'],
      dtype='object', length=153)

In [ ]:
# 결측치 값 확인을 편하게 하기 위해서 모두 np.nan 값으로 바꿔준다
# train_2017에 있는 모든 columns들을 불러온다.
for col in train_2019:
        train_2019[col].replace(' ', np.nan, inplace = True)
        if col != 'knowcode':
            test_2019[col].replace(' ', np.nan, inplace = True)

In [ ]:
# 정상적으로 결측치 값이 나타나는 것을 확인할 수 있습니다.
train_2019.isnull().sum()

idx            0
sq1            0
sq2            0
sq3            0
sq4            0
            ... 
bq30          23
bq31_1      1416
bq31_2      1503
bq31_3      7342
knowcode       0
Length: 153, dtype: int64

In [ ]:
test_2019.isnull().sum()

idx          0
sq1          0
sq2          0
sq3          0
sq4          0
          ... 
bq29      1228
bq30        25
bq31_1    1384
bq31_2    1478
bq31_3    7385
Length: 152, dtype: int64

In [ ]:
# idx와 knowcode를 제외하고는 154개의 특징들로 구성되어 있는 것을 확인할 수 있다.
train_2019.head()

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,kq6_1,kq6_2,kq7_1,kq7_2,kq8_1,kq8_2,kq9_1,kq9_2,kq10_1,kq10_2,kq11_1,kq11_2,kq12_1,...,bq14_5,bq15,bq16_1,bq16_2,bq16_3,bq16_4,bq16_5,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq22,bq23,bq24,bq25,bq26,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
0,18569,4,4,4,3,4,4,4,4,4,4,4,5,4,4,4,4,3,3,2,2,1,0,2,2,2,2,3,4,4,5,1,0,4,5,4,5,4,5,4,...,5.0,4.0,4.0,4.0,4.0,3.0,4.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,사람은 줄고 일은 많으니까 업무가 많다,3.0,4,수송수요가 늘어날다,4.0,4.0,3,NaN,NaN,NaN,1,35.0,4,기계공학과,1.0,1,NaN,1,40,3500,3000,NaN,812301
1,18570,4,3,4,4,4,4,3,4,5,4,3,5,4,4,1,2,4,6,3,4,4,5,5,5,3,4,5,6,1,0,1,0,1,0,1,0,1,0,1,...,3.0,3.0,4.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,1.0,3,운송시장이 큰 변화 없을듯해서,2.0,3.0,3,"컨테이너 대여사업,랜탈사업",NaN,NaN,1,63.0,3,경영학,2.0,NaN,5,NaN,40.0,NaN,NaN,5000,15201
2,18571,2,3,2,2,2,2,2,2,3,3,3,2,2,2,2,2,2,2,1,0,3,4,2,3,2,2,1,0,1,0,3,3,1,0,1,0,1,0,1,...,4.0,1.0,3.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,1.0,3,NaN,4.0,4.0,4,숙박업,NaN,NaN,2,62.0,2,NaN,2.0,NaN,6,NaN,30,NaN,NaN,3000,901101
3,18572,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,7,5,6,5,6,5,6,4,5,4,5,4,5,5,6,3,3,3,3,2,...,4.0,3.0,4.0,3.0,3.0,3.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,4,편리하게 일처리을 하기위해서 문서대행 업무를 맡길거 같아서,2.0,3.0,3,NaN,NaN,NaN,2,33.0,4,행정학,1.0,1,NaN,1,40.0,3600,2400,NaN,29903
4,18573,1,4,4,1,1,2,4,3,4,4,4,5,4,3,1,1,2,2,2,2,2,2,2,2,4,5,1,0,1,0,1,0,1,0,1,0,1,0,1,...,3.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,2,일거리가 줄어들고 있다,3.0,2.0,2,용역회사(청소원),NaN,NaN,1,67.0,1,NaN,2.0,NaN,6,NaN,45,NaN,NaN,1500,561401


### 1.2 데이터 타입 분포도 확인

In [ ]:
# 데이터의 타입 분포도는 int형이 104개, object형이 24개, float 64형이 23개 있는 것을
# 확인할 수 있다.

test_2019[test_2019.columns[1:]].dtypes.value_counts()

int64      104
object      24
float64     23
dtype: int64

In [ ]:
test_2019.select_dtypes('float64')

,bq13_1,bq13_3,bq14_1,bq14_2,bq14_3,bq14_5,bq15,bq16_1,bq16_2,bq16_3,bq16_4,bq16_5,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq21_2
0,2.0,3.0,3.0,4.0,4.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,4.0,2.0,3.0,4.0,4.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,3.0,4.0,5.0,6.0,6.0,3.0,2.0,1.0,2.0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
3,3.0,3.0,2.0,2.0,1.0,1.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,2.0,4.0,4.0,4.0,4.0,2.0,2.0,4.0,2.0,2.0,2.0,2.0,4.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8549,4.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,4.0,2.0,3.0,3.0,5.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,3.0
8550,2.0,4.0,5.0,5.0,6.0,4.0,2.0,3.0,2.0,3.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0
8551,2.0,3.0,4.0,5.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0
8552,1.0,3.0,4.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,4.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0


In [ ]:
for index, value, tindex, tvalue in zip(train_2019.isnull().sum().index, train_2019.isnull().sum(), test_2019.isnull().sum().index, test_2019.isnull().sum()):
    print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

idx : 0 / idx : 0 / 0
sq1 : 0 / sq1 : 0 / 0
sq2 : 0 / sq2 : 0 / 0
sq3 : 0 / sq3 : 0 / 0
sq4 : 0 / sq4 : 0 / 0
sq5 : 0 / sq5 : 0 / 0
sq6 : 0 / sq6 : 0 / 0
sq7 : 0 / sq7 : 0 / 0
sq8 : 0 / sq8 : 0 / 0
sq9 : 0 / sq9 : 0 / 0
sq10 : 0 / sq10 : 0 / 0
sq11 : 0 / sq11 : 0 / 0
sq12 : 0 / sq12 : 0 / 0
sq13 : 0 / sq13 : 0 / 0
sq14 : 0 / sq14 : 0 / 0
sq15 : 0 / sq15 : 0 / 0
sq16 : 0 / sq16 : 0 / 0
kq1_1 : 0 / kq1_1 : 0 / 0
kq1_2 : 0 / kq1_2 : 0 / 0
kq2_1 : 0 / kq2_1 : 0 / 0
kq2_2 : 0 / kq2_2 : 0 / 0
kq3_1 : 0 / kq3_1 : 0 / 0
kq3_2 : 0 / kq3_2 : 0 / 0
kq4_1 : 0 / kq4_1 : 0 / 0
kq4_2 : 0 / kq4_2 : 0 / 0
kq5_1 : 0 / kq5_1 : 0 / 0
kq5_2 : 0 / kq5_2 : 0 / 0
kq6_1 : 0 / kq6_1 : 0 / 0
kq6_2 : 0 / kq6_2 : 0 / 0
kq7_1 : 0 / kq7_1 : 0 / 0
kq7_2 : 0 / kq7_2 : 0 / 0
kq8_1 : 0 / kq8_1 : 0 / 0
kq8_2 : 0 / kq8_2 : 0 / 0
kq9_1 : 0 / kq9_1 : 0 / 0
kq9_2 : 0 / kq9_2 : 0 / 0
kq10_1 : 0 / kq10_1 : 0 / 0
kq10_2 : 0 / kq10_2 : 0 / 0
kq11_1 : 0 / kq11_1 : 0 / 0
kq11_2 : 0 / kq11_2 : 0 / 0
kq12_1 : 0 / kq12_1 : 0 / 0
kq12

## bq4_1a ~ 1c 자격증 종류
- bq4는 모두 응답했다.
- bq4가 1인 사람들 중에서 업무에 요구되는 자격증을 적지 않은 사람이 있다. 일단 2번을 선택한 모든 사람들의 자격증을 없음으로 지정한다.

In [ ]:
train_2019.bq4.value_counts()

2    4434
1    4121
Name: bq4, dtype: int64

In [ ]:
test_2019.bq4.value_counts()

2    4427
1    4127
Name: bq4, dtype: int64

In [ ]:
train_2019.loc[(train_2019['bq4'] == 2) & (train_2019['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
test_2019.loc[(test_2019['bq4'] == 2) & (test_2019['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
train_2019.loc[(train_2019['bq4'] == 2) & (train_2019['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
train_2019.loc[(train_2019['bq4'] == 2) & (train_2019['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
test_2019.loc[(test_2019['bq4'] == 2) & (test_2019['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
test_2019.loc[(test_2019['bq4'] == 2) & (test_2019['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

1번을 선택하고 자격증을 기입하지 않은 9명에 대해서는 가장 흔한 자격증인 '정보처리기사'로 표현하기로 함.

In [ ]:
train_2019.bq4_1a.value_counts()

없음                4434
정보처리기사              87
전기기사                81
의사면허증               65
토목기사                63
                  ... 
민간자격증                1
전기 산업기사              1
냉동시설안전관리사            1
환경기사(수질, 대기 등)       1
플로리스트자격증             1
Name: bq4_1a, Length: 1559, dtype: int64

In [ ]:
test_2019.bq4_1a.value_counts()

없음          4427
정보처리기사        94
전기기사          76
의사면허증         57
의사면허          52
            ... 
금융인증           1
자동차 면허증        1
소음진동기술사        1
농기계정비기능사       1
통신관련           1
Name: bq4_1a, Length: 1559, dtype: int64

In [ ]:
train_2019.loc[(train_2019['bq4'] == 1) & (train_2019['bq4_1a'].isnull()), 'bq4_1a'] = '정보처리기사'
test_2019.loc[(test_2019['bq4'] == 1) & (test_2019['bq4_1a'].isnull()), 'bq4_1a'] = '정보처리기사'

이제 나머지 자격증의 경우 없음으로 통일한다

In [ ]:
train_2019.loc[(train_2019['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
train_2019.loc[(train_2019['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
test_2019.loc[(test_2019['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
test_2019.loc[(test_2019['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

In [ ]:
for index, value, tindex, tvalue in zip(train_2019.isnull().sum().index, train_2019.isnull().sum(), test_2019.isnull().sum().index, test_2019.isnull().sum()):
    print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

idx : 0 / idx : 0 / 0
sq1 : 0 / sq1 : 0 / 0
sq2 : 0 / sq2 : 0 / 0
sq3 : 0 / sq3 : 0 / 0
sq4 : 0 / sq4 : 0 / 0
sq5 : 0 / sq5 : 0 / 0
sq6 : 0 / sq6 : 0 / 0
sq7 : 0 / sq7 : 0 / 0
sq8 : 0 / sq8 : 0 / 0
sq9 : 0 / sq9 : 0 / 0
sq10 : 0 / sq10 : 0 / 0
sq11 : 0 / sq11 : 0 / 0
sq12 : 0 / sq12 : 0 / 0
sq13 : 0 / sq13 : 0 / 0
sq14 : 0 / sq14 : 0 / 0
sq15 : 0 / sq15 : 0 / 0
sq16 : 0 / sq16 : 0 / 0
kq1_1 : 0 / kq1_1 : 0 / 0
kq1_2 : 0 / kq1_2 : 0 / 0
kq2_1 : 0 / kq2_1 : 0 / 0
kq2_2 : 0 / kq2_2 : 0 / 0
kq3_1 : 0 / kq3_1 : 0 / 0
kq3_2 : 0 / kq3_2 : 0 / 0
kq4_1 : 0 / kq4_1 : 0 / 0
kq4_2 : 0 / kq4_2 : 0 / 0
kq5_1 : 0 / kq5_1 : 0 / 0
kq5_2 : 0 / kq5_2 : 0 / 0
kq6_1 : 0 / kq6_1 : 0 / 0
kq6_2 : 0 / kq6_2 : 0 / 0
kq7_1 : 0 / kq7_1 : 0 / 0
kq7_2 : 0 / kq7_2 : 0 / 0
kq8_1 : 0 / kq8_1 : 0 / 0
kq8_2 : 0 / kq8_2 : 0 / 0
kq9_1 : 0 / kq9_1 : 0 / 0
kq9_2 : 0 / kq9_2 : 0 / 0
kq10_1 : 0 / kq10_1 : 0 / 0
kq10_2 : 0 / kq10_2 : 0 / 0
kq11_1 : 0 / kq11_1 : 0 / 0
kq11_2 : 0 / kq11_2 : 0 / 0
kq12_1 : 0 / kq12_1 : 0 / 0
kq12

bq5_1과 bq5_2는 2017년과 같이 처리

In [ ]:
train_2019.loc[(train_2019['bq5'] == 2) & (train_2019['bq5_1'].isnull()),'bq5_1'] = 0
test_2019.loc[(test_2019['bq5'] == 2) & (test_2019['bq5_1'].isnull()),'bq5_1'] = 0

In [ ]:
# 5번 문항에서 2번을 선택한 사람들에 대해서 5-2에 대한 값을 '없음'으로 처리함
train_2019.loc[(train_2019['bq5'] == 2) & (train_2019['bq5_2'].isnull()),'bq5_2'] = '없음'
test_2019.loc[(test_2019['bq5'] == 2) & (test_2019['bq5_2'].isnull()),'bq5_2'] = '없음'

In [ ]:
# bq5_2에서 많은 수가 남는다. 서술형이기 때문에 단순히 없음으로 처리하자.
train_2019.bq5_2.isnull().sum()

717

In [ ]:
train_2019.loc[train_2019['bq5_2'].isnull(), 'bq5_2'] = '없음'
test_2019.loc[test_2019['bq5_2'].isnull(), 'bq5_2'] = '없음'

## bq18_10, bq20_1, bq22, bq23, bq24

In [ ]:
#train_2019 = train_2019.drop(columns = ['bq18_10', 'bq20_1', 'bq22', 'bq23', 'bq24'])
#test_2019 = test_2019.drop(columns = ['bq18_10', 'bq20_1', 'bq22', 'bq23', 'bq24'])

In [ ]:
train_2019.loc[(train_2019['bq18_10'].isnull()), 'bq18_10'] = '없음'
train_2019.loc[(train_2019['bq20_1'].isnull()), 'bq20_1'] = '없음'
train_2019.loc[(train_2019['bq22'].isnull()), 'bq22'] = '없음'
train_2019.loc[(train_2019['bq23'].isnull()), 'bq23'] = '없음'
train_2019.loc[(train_2019['bq24'].isnull()), 'bq24'] = '없음'

In [ ]:
test_2019.loc[(test_2019['bq18_10'].isnull()), 'bq18_10'] = '없음'
test_2019.loc[(test_2019['bq20_1'].isnull()), 'bq20_1'] = '없음'
test_2019.loc[(test_2019['bq22'].isnull()), 'bq22'] = '없음'
test_2019.loc[(test_2019['bq23'].isnull()), 'bq23'] = '없음'
test_2019.loc[(test_2019['bq24'].isnull()), 'bq24'] = '없음'

In [ ]:
train_2019.bq18_10

0         사람은 줄고 일은 많으니까 업무가 많다
1                            없음
2                            없음
3                            없음
4                            없음
                 ...           
8550    방송장비의 기술변화로 장비도입 및 교육실시
8551                         없음
8552                         없음
8553                         없음
8554                         없음
Name: bq18_10, Length: 8555, dtype: object

## bq27_1은 2017년 bq38_1과 동일하게 처리합니다

In [ ]:
train_2019.loc[train_2019['bq27_1'].isnull(), 'bq27_1'] = '없음'
test_2019.loc[test_2019['bq27_1'].isnull(), 'bq27_1'] = '없음'

bq28에서 1을 답한 사람은 28_2가 NaN이고, 2를 답한 사람은 28_1이 NaN이다. 그러므로 그냥 두 개는 0으로 처리해준다.

In [ ]:
train_2019['bq28_1'].replace('없음', np.nan, inplace = True)
test_2019['bq28_1'].replace('없음', np.nan, inplace = True)
train_2019['bq28_2'].replace('없음', np.nan, inplace = True)
test_2019['bq28_2'].replace('없음', np.nan, inplace = True)

In [ ]:
train_2019['bq28_1'].replace(np.nan, 0,inplace = True)
test_2019['bq28_1'].replace(np.nan, 0,inplace = True)
train_2019['bq28_2'].replace(np.nan,0, inplace = True)
test_2019['bq28_2'].replace(np.nan, 0,inplace = True)

In [ ]:
test_2019.bq28_1.isnull().sum()

0

In [ ]:
train_2019['bq29'].replace(np.nan,0, inplace = True)
test_2019['bq29'].replace(np.nan, 0,inplace = True)

In [ ]:
for index, value, tindex, tvalue in zip(train_2019.isnull().sum().index, train_2019.isnull().sum(), test_2019.isnull().sum().index, test_2019.isnull().sum()):
    print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

idx : 0 / idx : 0 / 0
sq1 : 0 / sq1 : 0 / 0
sq2 : 0 / sq2 : 0 / 0
sq3 : 0 / sq3 : 0 / 0
sq4 : 0 / sq4 : 0 / 0
sq5 : 0 / sq5 : 0 / 0
sq6 : 0 / sq6 : 0 / 0
sq7 : 0 / sq7 : 0 / 0
sq8 : 0 / sq8 : 0 / 0
sq9 : 0 / sq9 : 0 / 0
sq10 : 0 / sq10 : 0 / 0
sq11 : 0 / sq11 : 0 / 0
sq12 : 0 / sq12 : 0 / 0
sq13 : 0 / sq13 : 0 / 0
sq14 : 0 / sq14 : 0 / 0
sq15 : 0 / sq15 : 0 / 0
sq16 : 0 / sq16 : 0 / 0
kq1_1 : 0 / kq1_1 : 0 / 0
kq1_2 : 0 / kq1_2 : 0 / 0
kq2_1 : 0 / kq2_1 : 0 / 0
kq2_2 : 0 / kq2_2 : 0 / 0
kq3_1 : 0 / kq3_1 : 0 / 0
kq3_2 : 0 / kq3_2 : 0 / 0
kq4_1 : 0 / kq4_1 : 0 / 0
kq4_2 : 0 / kq4_2 : 0 / 0
kq5_1 : 0 / kq5_1 : 0 / 0
kq5_2 : 0 / kq5_2 : 0 / 0
kq6_1 : 0 / kq6_1 : 0 / 0
kq6_2 : 0 / kq6_2 : 0 / 0
kq7_1 : 0 / kq7_1 : 0 / 0
kq7_2 : 0 / kq7_2 : 0 / 0
kq8_1 : 0 / kq8_1 : 0 / 0
kq8_2 : 0 / kq8_2 : 0 / 0
kq9_1 : 0 / kq9_1 : 0 / 0
kq9_2 : 0 / kq9_2 : 0 / 0
kq10_1 : 0 / kq10_1 : 0 / 0
kq10_2 : 0 / kq10_2 : 0 / 0
kq11_1 : 0 / kq11_1 : 0 / 0
kq11_2 : 0 / kq11_2 : 0 / 0
kq12_1 : 0 / kq12_1 : 0 / 0
kq12

## bq_31 1~3
- NaN값에 대해서는 각 bq1(근무하고 있는 산업)의 임금 및 사업소득 중위값을 채워넣자.

In [ ]:
total_2019 = pd.concat([train_2019, test_2019], axis=0)
total_2019.head()

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,kq6_1,kq6_2,kq7_1,kq7_2,kq8_1,kq8_2,kq9_1,kq9_2,kq10_1,kq10_2,kq11_1,kq11_2,kq12_1,...,bq14_5,bq15,bq16_1,bq16_2,bq16_3,bq16_4,bq16_5,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq22,bq23,bq24,bq25,bq26,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
0,18569,4,4,4,3,4,4,4,4,4,4,4,5,4,4,4,4,3,3,2,2,1,0,2,2,2,2,3,4,4,5,1,0,4,5,4,5,4,5,4,...,5.0,4.0,4.0,4.0,4.0,3.0,4.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,사람은 줄고 일은 많으니까 업무가 많다,3,4,수송수요가 늘어날다,4.0,4.0,3,없음,없음,없음,1,35,4,기계공학과,1,1,0,1,40,3500,3000,NaN,812301.0
1,18570,4,3,4,4,4,4,3,4,5,4,3,5,4,4,1,2,4,6,3,4,4,5,5,5,3,4,5,6,1,0,1,0,1,0,1,0,1,0,1,...,3.0,3.0,4.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,없음,1,3,운송시장이 큰 변화 없을듯해서,2.0,3.0,3,"컨테이너 대여사업,랜탈사업",없음,없음,1,63,3,경영학,2,0,5,0,40.0,NaN,NaN,5000,15201.0
2,18571,2,3,2,2,2,2,2,2,3,3,3,2,2,2,2,2,2,2,1,0,3,4,2,3,2,2,1,0,1,0,3,3,1,0,1,0,1,0,1,...,4.0,1.0,3.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,없음,1,3,없음,4.0,4.0,4,숙박업,없음,없음,2,62,2,없음,2,0,6,0,30,NaN,NaN,3000,901101.0
3,18572,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,7,5,6,5,6,5,6,4,5,4,5,4,5,5,6,3,3,3,3,2,...,4.0,3.0,4.0,3.0,3.0,3.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,없음,2,4,편리하게 일처리을 하기위해서 문서대행 업무를 맡길거 같아서,2.0,3.0,3,없음,없음,없음,2,33,4,행정학,1,1,0,1,40.0,3600,2400,NaN,29903.0
4,18573,1,4,4,1,1,2,4,3,4,4,4,5,4,3,1,1,2,2,2,2,2,2,2,2,4,5,1,0,1,0,1,0,1,0,1,0,1,0,1,...,3.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,없음,2,2,일거리가 줄어들고 있다,3.0,2.0,2,용역회사(청소원),없음,없음,1,67,1,없음,2,0,6,0,45,NaN,NaN,1500,561401.0


In [ ]:
bq1_list = total_2019.bq1.unique()
bq1_list

array([15,  8,  1, 19,  3, 17, 20,  6, 18, 10,  9, 11, 13, 16,  7,  4,  5,
        2, 14, 12, 21])

In [ ]:
for bq1 in bq1_list:
    mean_bq31_1 = round(total_2019.loc[(total_2019.bq1 == bq1) & (total_2019.bq31_1.notnull())].bq31_1.apply(lambda x: int(x)).mean())
    train_2019.loc[(train_2019.bq1 == bq1) & (train_2019.bq31_1.isnull()), 'bq31_1'] = mean_bq31_1
    test_2019.loc[(test_2019.bq1 == bq1) & (test_2019.bq31_1.isnull()), 'bq31_1'] = mean_bq31_1
    
    mean_bq31_2 = round(total_2019.loc[(total_2019.bq1 == bq1) & (total_2019.bq31_2.notnull())].bq31_2.apply(lambda x: int(x)).mean())
    
    try:
        mean_bq31_3 = round(total_2019.loc[(total_2019.bq1 == bq1) & (total_2019.bq31_3.notnull())].bq31_3.apply(lambda x: int(x)).mean())
    except:
        mean_bq31_3 = mean_bq31_2
    
    train_2019.loc[(train_2019.bq1 == bq1) & (train_2019.bq31_2.isnull()), 'bq31_2'] = mean_bq31_2
    test_2019.loc[(test_2019.bq1 == bq1) & (test_2019.bq31_2.isnull()), 'bq31_2'] = mean_bq31_2
    
    train_2019.loc[(train_2019.bq1 == bq1) & (train_2019.bq31_3.isnull()), 'bq31_3'] = mean_bq31_3
    test_2019.loc[(test_2019.bq1 == bq1) & (test_2019.bq31_3.isnull()), 'bq31_3'] = mean_bq31_3

In [ ]:
for index, value, tindex, tvalue in zip(train_2019.isnull().sum().index, train_2019.isnull().sum(), test_2019.isnull().sum().index, test_2019.isnull().sum()):
    print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

idx : 0 / idx : 0 / 0
sq1 : 0 / sq1 : 0 / 0
sq2 : 0 / sq2 : 0 / 0
sq3 : 0 / sq3 : 0 / 0
sq4 : 0 / sq4 : 0 / 0
sq5 : 0 / sq5 : 0 / 0
sq6 : 0 / sq6 : 0 / 0
sq7 : 0 / sq7 : 0 / 0
sq8 : 0 / sq8 : 0 / 0
sq9 : 0 / sq9 : 0 / 0
sq10 : 0 / sq10 : 0 / 0
sq11 : 0 / sq11 : 0 / 0
sq12 : 0 / sq12 : 0 / 0
sq13 : 0 / sq13 : 0 / 0
sq14 : 0 / sq14 : 0 / 0
sq15 : 0 / sq15 : 0 / 0
sq16 : 0 / sq16 : 0 / 0
kq1_1 : 0 / kq1_1 : 0 / 0
kq1_2 : 0 / kq1_2 : 0 / 0
kq2_1 : 0 / kq2_1 : 0 / 0
kq2_2 : 0 / kq2_2 : 0 / 0
kq3_1 : 0 / kq3_1 : 0 / 0
kq3_2 : 0 / kq3_2 : 0 / 0
kq4_1 : 0 / kq4_1 : 0 / 0
kq4_2 : 0 / kq4_2 : 0 / 0
kq5_1 : 0 / kq5_1 : 0 / 0
kq5_2 : 0 / kq5_2 : 0 / 0
kq6_1 : 0 / kq6_1 : 0 / 0
kq6_2 : 0 / kq6_2 : 0 / 0
kq7_1 : 0 / kq7_1 : 0 / 0
kq7_2 : 0 / kq7_2 : 0 / 0
kq8_1 : 0 / kq8_1 : 0 / 0
kq8_2 : 0 / kq8_2 : 0 / 0
kq9_1 : 0 / kq9_1 : 0 / 0
kq9_2 : 0 / kq9_2 : 0 / 0
kq10_1 : 0 / kq10_1 : 0 / 0
kq10_2 : 0 / kq10_2 : 0 / 0
kq11_1 : 0 / kq11_1 : 0 / 0
kq11_2 : 0 / kq11_2 : 0 / 0
kq12_1 : 0 / kq12_1 : 0 / 0
kq12

## 나머지
나머지 값들은 각 항목들의 최빈값을 구하여 넣어주도록 하겠다

In [ ]:
from sklearn.impute import SimpleImputer
train_columns = train_2019.columns
test_columns = test_2019.columns
train_target = train_2019.iloc[:,-1]
imp = SimpleImputer(strategy='most_frequent')
train_2019_no_target = pd.DataFrame(imp.fit_transform(train_2019.iloc[:,:-1]))
test_2019 = pd.DataFrame(imp.transform(test_2019))
train_2019 = pd.concat([train_2019_no_target, train_target], axis=1)

train_2019.columns = train_columns
test_2019.columns = test_columns


In [ ]:
train_2019.head()

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,kq6_1,kq6_2,kq7_1,kq7_2,kq8_1,kq8_2,kq9_1,kq9_2,kq10_1,kq10_2,kq11_1,kq11_2,kq12_1,...,bq14_5,bq15,bq16_1,bq16_2,bq16_3,bq16_4,bq16_5,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq22,bq23,bq24,bq25,bq26,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
0,18569,4,4,4,3,4,4,4,4,4,4,4,5,4,4,4,4,3,3,2,2,1,0,2,2,2,2,3,4,4,5,1,0,4,5,4,5,4,5,4,...,5,4,4,4,4,3,4,2,1,1,1,1,2,2,1,1,2,사람은 줄고 일은 많으니까 업무가 많다,3,4,수송수요가 늘어날다,4,4,3,없음,없음,없음,1,35,4,기계공학과,1,1,0,1,40,3500,3000,5500,812301
1,18570,4,3,4,4,4,4,3,4,5,4,3,5,4,4,1,2,4,6,3,4,4,5,5,5,3,4,5,6,1,0,1,0,1,0,1,0,1,0,1,...,3,3,4,2,3,2,2,3,2,2,2,2,2,2,2,2,2,없음,1,3,운송시장이 큰 변화 없을듯해서,2,3,3,"컨테이너 대여사업,랜탈사업",없음,없음,1,63,3,경영학,2,0,5,0,40.0,4843,3128,5000,15201
2,18571,2,3,2,2,2,2,2,2,3,3,3,2,2,2,2,2,2,2,1,0,3,4,2,3,2,2,1,0,1,0,3,3,1,0,1,0,1,0,1,...,4,1,3,2,2,2,2,1,2,2,2,2,2,2,2,2,2,없음,1,3,없음,4,4,4,숙박업,없음,없음,2,62,2,없음,2,0,6,0,30,3293,2104,3000,901101
3,18572,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,7,5,6,5,6,5,6,4,5,4,5,4,5,5,6,3,3,3,3,2,...,4,3,4,3,3,3,3,4,2,2,2,2,2,2,2,2,2,없음,2,4,편리하게 일처리을 하기위해서 문서대행 업무를 맡길거 같아서,2,3,3,없음,없음,없음,2,33,4,행정학,1,1,0,1,40.0,3600,2400,5500,29903
4,18573,1,4,4,1,1,2,4,3,4,4,4,5,4,3,1,1,2,2,2,2,2,2,2,2,4,5,1,0,1,0,1,0,1,0,1,0,1,0,1,...,3,3,2,2,2,2,2,3,2,2,2,2,2,2,2,2,2,없음,2,2,일거리가 줄어들고 있다,3,2,2,용역회사(청소원),없음,없음,1,67,1,없음,2,0,6,0,45,3143,2254,1500,561401


In [ ]:
test_2019.head()

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,kq6_1,kq6_2,kq7_1,kq7_2,kq8_1,kq8_2,kq9_1,kq9_2,kq10_1,kq10_2,kq11_1,kq11_2,kq12_1,...,bq14_4,bq14_5,bq15,bq16_1,bq16_2,bq16_3,bq16_4,bq16_5,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq22,bq23,bq24,bq25,bq26,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
0,18569,5,5,5,4,4,5,4,5,5,4,5,4,4,4,3,4,2,1,2,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,...,4,3,3,3,3,2,3,3,4,2,2,2,2,2,2,2,2,2,없음,1,4,생명과 관련된 일이므로,2,2,2,사무직,없음,없음,1,57,4,경영,1,1,0,1,45,6700,3800,5500
1,18570,2,3,3,1,3,2,3,3,3,2,3,4,3,2,1,3,1,0,1,0,1,0,1,0,1,0,1,0,3,3,1,0,1,0,1,0,1,0,1,...,5,2,2,1,2,1,1,2,3,2,2,2,2,2,2,2,2,2,없음,2,3,3D업종이라서,2,2,2,없음,없음,없음,1,27,3,화학,1,1,0,1,44,2700,2500,3484
2,18571,5,4,5,4,4,4,4,5,4,4,4,5,4,4,4,4,2,2,3,3,1,0,2,3,1,0,1,0,1,0,1,0,3,4,3,4,1,0,4,...,5,3,2,1,2,1,2,3,2,2,2,2,2,2,2,2,2,2,없음,1,4,산업안전 법규등의 강화로 안전관리자의 역활이 요구될것이므로,2,1,2,없음,없음,없음,1,40,4,건축공학,1,1,0,1,40,6000,3200,4292
3,18572,3,3,3,2,3,3,3,3,2,2,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,0,1,0,1,0,1,0,1,0,1,...,1,1,3,3,3,3,4,4,4,2,2,2,2,2,2,2,2,2,없음,2,5,차량증가,2,2,2,주부,없음,없음,2,44,2,인문,1,1,0,1,60,900,800,5449
4,18573,5,4,5,3,3,3,3,4,5,4,5,4,5,3,5,4,2,4,4,4,1,0,1,0,2,3,1,0,5,5,1,0,5,6,5,6,3,5,1,...,1,2,2,4,2,2,2,2,4,1,1,1,1,2,2,2,2,1,"인공지능, 4차산업혁명 등 새로운 R&D분야 필요, 연구개발 요구 받음, 혁신적인 ...",2,4,"무인화, 자동화 등 새로운 장비 기술에 대한 연구개발 수요 증가",1,4,3,자동차생산기술담당,없음,없음,1,36,6,기계공학과,1,1,0,1,50,7000,5000,7265


In [ ]:
for index, value, tindex, tvalue in zip(train_2019.isnull().sum().index, train_2019.isnull().sum(), test_2019.isnull().sum().index, test_2019.isnull().sum()):
    print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

idx : 0 / idx : 0 / 0
sq1 : 0 / sq1 : 0 / 0
sq2 : 0 / sq2 : 0 / 0
sq3 : 0 / sq3 : 0 / 0
sq4 : 0 / sq4 : 0 / 0
sq5 : 0 / sq5 : 0 / 0
sq6 : 0 / sq6 : 0 / 0
sq7 : 0 / sq7 : 0 / 0
sq8 : 0 / sq8 : 0 / 0
sq9 : 0 / sq9 : 0 / 0
sq10 : 0 / sq10 : 0 / 0
sq11 : 0 / sq11 : 0 / 0
sq12 : 0 / sq12 : 0 / 0
sq13 : 0 / sq13 : 0 / 0
sq14 : 0 / sq14 : 0 / 0
sq15 : 0 / sq15 : 0 / 0
sq16 : 0 / sq16 : 0 / 0
kq1_1 : 0 / kq1_1 : 0 / 0
kq1_2 : 0 / kq1_2 : 0 / 0
kq2_1 : 0 / kq2_1 : 0 / 0
kq2_2 : 0 / kq2_2 : 0 / 0
kq3_1 : 0 / kq3_1 : 0 / 0
kq3_2 : 0 / kq3_2 : 0 / 0
kq4_1 : 0 / kq4_1 : 0 / 0
kq4_2 : 0 / kq4_2 : 0 / 0
kq5_1 : 0 / kq5_1 : 0 / 0
kq5_2 : 0 / kq5_2 : 0 / 0
kq6_1 : 0 / kq6_1 : 0 / 0
kq6_2 : 0 / kq6_2 : 0 / 0
kq7_1 : 0 / kq7_1 : 0 / 0
kq7_2 : 0 / kq7_2 : 0 / 0
kq8_1 : 0 / kq8_1 : 0 / 0
kq8_2 : 0 / kq8_2 : 0 / 0
kq9_1 : 0 / kq9_1 : 0 / 0
kq9_2 : 0 / kq9_2 : 0 / 0
kq10_1 : 0 / kq10_1 : 0 / 0
kq10_2 : 0 / kq10_2 : 0 / 0
kq11_1 : 0 / kq11_1 : 0 / 0
kq11_2 : 0 / kq11_2 : 0 / 0
kq12_1 : 0 / kq12_1 : 0 / 0
kq12

## 저장

In [ ]:
train_2019.to_csv('/content/gdrive/MyDrive/KNOW_data/train/train_2019_서술형살림.csv')
test_2019.to_csv('/content/gdrive/MyDrive/KNOW_data/test/test_2019_서술형살림.csv')

# 2020년 데이터

In [ ]:
train_2020.isnull().sum()

idx         0
saq1_1      0
saq1_2      0
saq2_1      0
saq2_2      0
           ..
bq29        0
bq30_1      0
bq30_2      0
bq30_3      0
knowcode    0
Length: 185, dtype: int64

In [ ]:
for col in train_2020:
        train_2020[col].replace(' ', np.nan, inplace = True)
        if col != 'knowcode':
            test_2020[col].replace(' ', np.nan, inplace = True)

In [ ]:
train_2020.isnull().sum()

idx            0
saq1_1         0
saq1_2       676
saq2_1         0
saq2_2       457
            ... 
bq29           0
bq30_1      1016
bq30_2      1018
bq30_3      7104
knowcode       0
Length: 185, dtype: int64

In [ ]:
train_2020.head(1)

,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,saq4_2,saq5_1,saq5_2,saq6_1,saq6_2,saq7_1,saq7_2,saq8_1,saq8_2,saq9_1,saq9_2,saq10_1,saq10_2,saq11_1,saq11_2,saq12_1,saq12_2,saq13_1,saq13_2,saq14_1,saq14_2,saq15_1,saq15_2,saq16_1,saq16_2,saq17_1,saq17_2,saq18_1,saq18_2,saq19_1,saq19_2,saq20_1,...,bq16_10,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq21_4,bq22_1,bq22_2,bq22_3,bq22_4,bq22_5,bq22_6,bq23_1,bq23_2,bq23_3,bq24,bq25,bq26,bq26_1,bq27_1,bq27_2,bq28,bq29,bq30_1,bq30_2,bq30_3,knowcode
0,27127,4,4,4,4,4,5,4,5,3,3,4,5,1,NaN,4,5,3,4,1,NaN,3,4,3,4,4,5,3,4,4,5,3,4,3,4,1,NaN,1,NaN,4,...,4,4,2,2,2,2,2,2,2,2,2,NaN,1,3,일정하게 인원이 정해져 있으므로 일자리 변화는 없을 것 같아서,3,2,3,2,5,7,4,5,2,3,3,4,3,2,42,4,간호학,1,1,1,40,4000,2600,NaN,304003


In [ ]:
test_2020[test_2020.columns[1:]].dtypes.value_counts()

int64     127
object     56
dtype: int64

In [ ]:
for index, value, tindex, tvalue in zip(train_2020.isnull().sum().index, train_2020.isnull().sum(), test_2020.isnull().sum().index, test_2020.isnull().sum()):
    if (value + tvalue) != 0:
        print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

saq1_2 : 676 / saq1_2 : 683 / 1359
saq2_2 : 457 / saq2_2 : 456 / 913
saq3_2 : 1606 / saq3_2 : 1618 / 3224
saq4_2 : 836 / saq4_2 : 890 / 1726
saq5_2 : 1705 / saq5_2 : 1689 / 3394
saq6_2 : 1312 / saq6_2 : 1342 / 2654
saq7_2 : 2175 / saq7_2 : 2148 / 4323
saq8_2 : 845 / saq8_2 : 873 / 1718
saq9_2 : 531 / saq9_2 : 595 / 1126
saq10_2 : 1546 / saq10_2 : 1554 / 3100
saq11_2 : 1821 / saq11_2 : 1786 / 3607
saq12_2 : 1297 / saq12_2 : 1353 / 2650
saq13_2 : 515 / saq13_2 : 537 / 1052
saq14_2 : 1490 / saq14_2 : 1506 / 2996
saq15_2 : 1378 / saq15_2 : 1384 / 2762
saq16_2 : 1118 / saq16_2 : 1165 / 2283
saq17_2 : 2006 / saq17_2 : 2034 / 4040
saq18_2 : 2356 / saq18_2 : 2323 / 4679
saq19_2 : 1591 / saq19_2 : 1671 / 3262
saq20_2 : 1159 / saq20_2 : 1179 / 2338
saq21_2 : 676 / saq21_2 : 689 / 1365
saq22_2 : 1042 / saq22_2 : 1079 / 2121
saq23_2 : 878 / saq23_2 : 911 / 1789
saq24_2 : 2700 / saq24_2 : 2727 / 5427
saq25_2 : 2252 / saq25_2 : 2245 / 4497
saq26_2 : 2651 / saq26_2 : 2646 / 5297
saq27_2 : 2144 / saq2

In [ ]:
for i in range(1, 45):
    if len(train_2020.loc[lambda x : (x['saq'+str(i)+'_2'].notnull()) & (x['saq'+str(i)+'_1'] == 1)]) != 0:
        print(i)

40


In [ ]:
train_2020.loc[lambda x : (x['saq'+str(40)+'_2'].notnull()) & (x['saq'+str(40)+'_1'] == 1)]

,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,saq4_2,saq5_1,saq5_2,saq6_1,saq6_2,saq7_1,saq7_2,saq8_1,saq8_2,saq9_1,saq9_2,saq10_1,saq10_2,saq11_1,saq11_2,saq12_1,saq12_2,saq13_1,saq13_2,saq14_1,saq14_2,saq15_1,saq15_2,saq16_1,saq16_2,saq17_1,saq17_2,saq18_1,saq18_2,saq19_1,saq19_2,saq20_1,...,bq16_10,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq21_4,bq22_1,bq22_2,bq22_3,bq22_4,bq22_5,bq22_6,bq23_1,bq23_2,bq23_3,bq24,bq25,bq26,bq26_1,bq27_1,bq27_2,bq28,bq29,bq30_1,bq30_2,bq30_3,knowcode
2177,29304,3,7,3,6,3,5,3,6,3,6,3,6,3,6,2,5,2,5,3,4,3,5,3,6,3,6,3,5,3,5,3,5,3,5,2,4,3,6,3,...,5,5,2,2,2,2,2,2,2,2,2,NaN,1,3,경영을 지원하는 사무 인력을 최소화하는 경향이 있어 약간의 감소,2,2,2,1,7,7,7,5,4,4,6,4,5,1,43,4,경영학,1,1,1,48,4500,2500,NaN,26301


In [ ]:
train_2020.loc[lambda x : (x['saq'+str(40)+'_2'].notnull()) & (x['saq'+str(40)+'_1'] == 1), 'saq40_2'] = np.nan

In [ ]:
train_2020.loc[lambda x : (x['saq'+str(40)+'_2'].notnull()) & (x['saq'+str(40)+'_1'] == 1)]

,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,saq4_2,saq5_1,saq5_2,saq6_1,saq6_2,saq7_1,saq7_2,saq8_1,saq8_2,saq9_1,saq9_2,saq10_1,saq10_2,saq11_1,saq11_2,saq12_1,saq12_2,saq13_1,saq13_2,saq14_1,saq14_2,saq15_1,saq15_2,saq16_1,saq16_2,saq17_1,saq17_2,saq18_1,saq18_2,saq19_1,saq19_2,saq20_1,...,bq16_10,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq21_4,bq22_1,bq22_2,bq22_3,bq22_4,bq22_5,bq22_6,bq23_1,bq23_2,bq23_3,bq24,bq25,bq26,bq26_1,bq27_1,bq27_2,bq28,bq29,bq30_1,bq30_2,bq30_3,knowcode


In [ ]:
for i in range(1, 45):
    if len(train_2020.loc[lambda x : (x['saq'+str(i)+'_2'].isnull()) & (x['saq'+str(i)+'_1'] != 1)]) != 0:
        print(i)

In [ ]:
for i in range(1, 45):
    train_2020.loc[lambda x : (x['saq'+str(i)+'_2'].isnull()) & (x['saq'+str(i)+'_1'] == 1), 'saq'+str(i)+'_2'] = 0

In [ ]:
for i in range(1, 45):
    if len(test_2020.loc[lambda x : (x['saq'+str(i)+'_2'].notnull()) & (x['saq'+str(i)+'_1'] == 1)]) != 0:
        print(i)

In [ ]:
for i in range(1, 45):
    if len(test_2020.loc[lambda x : (x['saq'+str(i)+'_2'].isnull()) & (x['saq'+str(i)+'_1'] != 1)]) != 0:
        print(i)

40


In [ ]:
test_2020.loc[lambda x : (x['saq'+str(40)+'_2'].isnull()) & (x['saq'+str(40)+'_1'] != 1)]

,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,saq4_2,saq5_1,saq5_2,saq6_1,saq6_2,saq7_1,saq7_2,saq8_1,saq8_2,saq9_1,saq9_2,saq10_1,saq10_2,saq11_1,saq11_2,saq12_1,saq12_2,saq13_1,saq13_2,saq14_1,saq14_2,saq15_1,saq15_2,saq16_1,saq16_2,saq17_1,saq17_2,saq18_1,saq18_2,saq19_1,saq19_2,saq20_1,...,bq16_9,bq16_10,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq21_4,bq22_1,bq22_2,bq22_3,bq22_4,bq22_5,bq22_6,bq23_1,bq23_2,bq23_3,bq24,bq25,bq26,bq26_1,bq27_1,bq27_2,bq28,bq29,bq30_1,bq30_2,bq30_3
7136,34263,4,3,3,4,4,4,4,4,4,3,3,5,3,5,3,4,4,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,5,3,...,3,4,2,2,2,2,2,2,2,2,2,2,NaN,2,3,변화가 없어서,3,3,2,2,5,5,4,5,4,5,7,7,7,1,39,4,경영학,1,1,1,45,4500,3500,NaN


In [ ]:
test_2020.saq40_2.value_counts()

4    1940
3    1497
5    1326
2     816
6     301
1     122
7      32
Name: saq40_2, dtype: int64

In [ ]:
test_2020.loc[lambda x : (x['saq'+str(40)+'_2'].isnull()) & (x['saq'+str(40)+'_1'] != 1), 'saq40_2'] = 4

In [ ]:
test_2020.loc[lambda x : (x['saq'+str(40)+'_2'].isnull()) & (x['saq'+str(40)+'_1'] != 1)]

,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,saq4_2,saq5_1,saq5_2,saq6_1,saq6_2,saq7_1,saq7_2,saq8_1,saq8_2,saq9_1,saq9_2,saq10_1,saq10_2,saq11_1,saq11_2,saq12_1,saq12_2,saq13_1,saq13_2,saq14_1,saq14_2,saq15_1,saq15_2,saq16_1,saq16_2,saq17_1,saq17_2,saq18_1,saq18_2,saq19_1,saq19_2,saq20_1,...,bq16_9,bq16_10,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq21_4,bq22_1,bq22_2,bq22_3,bq22_4,bq22_5,bq22_6,bq23_1,bq23_2,bq23_3,bq24,bq25,bq26,bq26_1,bq27_1,bq27_2,bq28,bq29,bq30_1,bq30_2,bq30_3


In [ ]:
for i in range(1, 45):
    test_2020.loc[lambda x : (x['saq'+str(i)+'_2'].isnull()) & (x['saq'+str(i)+'_1'] == 1), 'saq'+str(i)+'_2'] = 0

In [ ]:
for index, value, tindex, tvalue in zip(train_2020.isnull().sum().index, train_2020.isnull().sum(), test_2020.isnull().sum().index, test_2020.isnull().sum()):
    if (value + tvalue) != 0:
        print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

bq4_1a : 3847 / bq4_1a : 3881 / 7728
bq4_1b : 6958 / bq4_1b : 7059 / 14017
bq4_1c : 7878 / bq4_1c : 7888 / 15766
bq5_1 : 5816 / bq5_1 : 5822 / 11638
bq5_2 : 5816 / bq5_2 : 5822 / 11638
bq18_10 : 6693 / bq18_10 : 6742 / 13435
bq26_1 : 134 / bq26_1 : 156 / 290
bq28 : 2 / bq28 : 6 / 8
bq30_1 : 1016 / bq30_1 : 1014 / 2030
bq30_2 : 1018 / bq30_2 : 1014 / 2032
bq30_3 : 7104 / bq30_3 : 7108 / 14212


In [ ]:
train_2020.loc[(train_2020['bq4'] == 2) & (train_2020['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
test_2020.loc[(test_2020['bq4'] == 2) & (test_2020['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
train_2020.loc[(train_2020['bq4'] == 2) & (train_2020['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
train_2020.loc[(train_2020['bq4'] == 2) & (train_2020['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
test_2020.loc[(test_2020['bq4'] == 2) & (test_2020['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
test_2020.loc[(test_2020['bq4'] == 2) & (test_2020['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

In [ ]:
train_2020.loc[lambda x: (x.bq4 ==1) & (x.bq4_1a.isnull())]

,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,saq4_2,saq5_1,saq5_2,saq6_1,saq6_2,saq7_1,saq7_2,saq8_1,saq8_2,saq9_1,saq9_2,saq10_1,saq10_2,saq11_1,saq11_2,saq12_1,saq12_2,saq13_1,saq13_2,saq14_1,saq14_2,saq15_1,saq15_2,saq16_1,saq16_2,saq17_1,saq17_2,saq18_1,saq18_2,saq19_1,saq19_2,saq20_1,...,bq16_10,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq21_4,bq22_1,bq22_2,bq22_3,bq22_4,bq22_5,bq22_6,bq23_1,bq23_2,bq23_3,bq24,bq25,bq26,bq26_1,bq27_1,bq27_2,bq28,bq29,bq30_1,bq30_2,bq30_3,knowcode


In [ ]:
train_2020.loc[(train_2020['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
train_2020.loc[(train_2020['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
test_2020.loc[(test_2020['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
test_2020.loc[(test_2020['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

In [ ]:
for col in test_2020:
    train_2020[col].replace('없다','없음', inplace = True)
    test_2020[col].replace('없다','없음', inplace = True)    

In [ ]:
for index, value, tindex, tvalue in zip(train_2020.isnull().sum().index, train_2020.isnull().sum(), test_2020.isnull().sum().index, test_2020.isnull().sum()):
    if (value + tvalue) != 0:
        print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

bq5_1 : 5816 / bq5_1 : 5822 / 11638
bq5_2 : 5816 / bq5_2 : 5822 / 11638
bq18_10 : 6693 / bq18_10 : 6742 / 13435
bq26_1 : 134 / bq26_1 : 156 / 290
bq28 : 2 / bq28 : 6 / 8
bq30_1 : 1016 / bq30_1 : 1014 / 2030
bq30_2 : 1018 / bq30_2 : 1014 / 2032
bq30_3 : 7104 / bq30_3 : 7108 / 14212


In [ ]:
print("학습 데이터 결측치 : ", train_2020[(train_2020['bq5'] == 2) & (train_2020['bq5_1'].isnull())].shape)
print("테스트 데이터 결측치 : ", test_2020[(test_2020['bq5'] == 2) & (test_2020['bq5_1'].isnull())].shape)

학습 데이터 결측치 :  (5816, 185)
테스트 데이터 결측치 :  (5822, 184)


In [ ]:
print("학습 데이터 결측치 : ", train_2020[(train_2020['bq5'] == 2) & (train_2020['bq5_2'].isnull())].shape)
print("테스트 데이터 결측치 : ", test_2020[(test_2020['bq5'] == 2) & (test_2020['bq5_2'].isnull())].shape)

학습 데이터 결측치 :  (5816, 185)
테스트 데이터 결측치 :  (5822, 184)


In [ ]:
train_2020.loc[(train_2020['bq5'] == 2) & (train_2020['bq5_1'].isnull()),'bq5_1'] = 0
test_2020.loc[(test_2020['bq5'] == 2) & (test_2020['bq5_1'].isnull()),'bq5_1'] = 0

In [ ]:
train_2020.loc[(train_2020['bq5'] == 2) & (train_2020['bq5_2'].isnull()),'bq5_2'] = '없음'
test_2020.loc[(test_2020['bq5'] == 2) & (test_2020['bq5_2'].isnull()),'bq5_2'] = '없음'

In [ ]:
print(train_2020.bq5_1.isnull().sum())
print(test_2020.bq5_1.isnull().sum())
print(train_2020.bq5_2.isnull().sum())
print(test_2020.bq5_2.isnull().sum())

0
0
0
0


In [ ]:
for index, value, tindex, tvalue in zip(train_2020.isnull().sum().index, train_2020.isnull().sum(), test_2020.isnull().sum().index, test_2020.isnull().sum()):
    if (value + tvalue) != 0:
        print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

bq18_10 : 6693 / bq18_10 : 6742 / 13435
bq26_1 : 134 / bq26_1 : 156 / 290
bq28 : 2 / bq28 : 6 / 8
bq30_1 : 1016 / bq30_1 : 1014 / 2030
bq30_2 : 1018 / bq30_2 : 1014 / 2032
bq30_3 : 7104 / bq30_3 : 7108 / 14212


In [ ]:
train_2020 = train_2020.drop(columns = ['bq18_10'])
test_2020 = test_2020.drop(columns = ['bq18_10'])

In [ ]:
print(len(train_2020.loc[lambda x : (x.bq26 != 1) & (x.bq26_1.isnull())]))
print(len(test_2020.loc[lambda x : (x.bq26 != 1) & (x.bq26_1.isnull())]))

0
0


In [ ]:
train_2020.loc[lambda x : x.bq26_1.isnull(), 'bq26_1'] = '없음'
test_2020.loc[lambda x : x.bq26_1.isnull(), 'bq26_1'] = '없음'

In [ ]:
for index, value, tindex, tvalue in zip(train_2020.isnull().sum().index, train_2020.isnull().sum(), test_2020.isnull().sum().index, test_2020.isnull().sum()):
    if (value + tvalue) != 0:
        print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

bq28 : 2 / bq28 : 6 / 8
bq30_1 : 1016 / bq30_1 : 1014 / 2030
bq30_2 : 1018 / bq30_2 : 1014 / 2032
bq30_3 : 7104 / bq30_3 : 7108 / 14212


In [ ]:
train_2020.loc[lambda x: x.bq28.isnull()][['bq27_1', 'bq27_2']]

,bq27_1,bq27_2
1838,1,4
3420,2,5


In [ ]:
test_2020.loc[lambda x: x.bq28.isnull()][['bq27_1', 'bq27_2']]

,bq27_1,bq27_2
1062,2,5
1789,1,4
4714,1,4
6401,2,5
7460,2,6
7939,2,6


In [ ]:
train_2020.loc[lambda x : (x.bq28.isnull()) & (x.bq27_1 == 1), 'bq28'] = 2
test_2020.loc[lambda x : (x.bq28.isnull()) & (x.bq27_1 == 1), 'bq28'] = 2
train_2020.loc[lambda x : (x.bq28.isnull()) & (x.bq27_1 == 2), 'bq28'] = 1
test_2020.loc[lambda x : (x.bq28.isnull()) & (x.bq27_1 == 2), 'bq28'] = 1

In [ ]:
for index, value, tindex, tvalue in zip(train_2020.isnull().sum().index, train_2020.isnull().sum(), test_2020.isnull().sum().index, test_2020.isnull().sum()):
    if (value + tvalue) != 0:
        print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

bq30_1 : 1016 / bq30_1 : 1014 / 2030
bq30_2 : 1018 / bq30_2 : 1014 / 2032
bq30_3 : 7104 / bq30_3 : 7108 / 14212


In [ ]:
total_2020 = pd.concat([train_2020, test_2020], axis=0)
total_2020.head()

,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,saq4_2,saq5_1,saq5_2,saq6_1,saq6_2,saq7_1,saq7_2,saq8_1,saq8_2,saq9_1,saq9_2,saq10_1,saq10_2,saq11_1,saq11_2,saq12_1,saq12_2,saq13_1,saq13_2,saq14_1,saq14_2,saq15_1,saq15_2,saq16_1,saq16_2,saq17_1,saq17_2,saq18_1,saq18_2,saq19_1,saq19_2,saq20_1,...,bq16_9,bq16_10,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq21_4,bq22_1,bq22_2,bq22_3,bq22_4,bq22_5,bq22_6,bq23_1,bq23_2,bq23_3,bq24,bq25,bq26,bq26_1,bq27_1,bq27_2,bq28,bq29,bq30_1,bq30_2,bq30_3,knowcode
0,27127,4,4,4,4,4,5,4,5,3,3,4,5,1,0,4,5,3,4,1,0,3,4,3,4,4,5,3,4,4,5,3,4,3,4,1,0,1,0,4,...,3,4,4,2,2,2,2,2,2,2,2,2,1,3,일정하게 인원이 정해져 있으므로 일자리 변화는 없을 것 같아서,3,2,3,2,5,7,4,5,2,3,3,4,3,2,42,4,간호학,1,1,1,40,4000,2600,NaN,304003.0
1,27128,5,6,5,6,4,5,4,5,4,5,4,5,5,6,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,3,4,4,...,3,4,3,2,2,2,2,2,2,2,2,2,1,2,경기 침체,3,3,3,2,3,4,3,5,6,6,3,6,5,1,45,4,기계공학,1,1,1,40,4000,3000,NaN,221401.0
2,27129,3,4,3,4,3,4,3,5,3,5,3,5,3,5,4,5,4,5,4,5,3,4,3,4,3,4,4,5,4,4,4,5,3,4,4,5,4,5,3,...,4,4,3,2,2,1,1,2,2,2,2,2,2,2,전체적으로 경기 악화 때문,3,2,3,2,5,7,5,5,5,5,5,5,5,1,38,4,기계,1,1,1,48,4300,3000,NaN,815001.0
3,27130,4,5,2,3,3,3,4,5,3,4,3,5,4,4,3,3,3,2,1,0,2,3,3,4,3,5,3,2,3,3,4,4,3,4,1,0,5,6,3,...,3,2,2,2,2,2,2,2,2,2,2,2,1,3,사람의 상태를 소통하며 확인해서 그에 맞는 각가의 처방을 내려줘야 하는 일이라 대체불가,3,3,3,2,6,4,7,3,6,7,7,6,4,2,25,4,보건학,1,1,1,40,3500,3300,NaN,304003.0
4,27131,5,6,4,5,3,5,4,4,4,6,4,5,4,5,5,6,4,5,4,5,4,5,4,5,5,6,4,5,1,0,1,0,4,5,4,5,4,5,5,...,3,5,4,2,2,2,2,2,2,2,2,2,2,2,건설경기가 아직은 침체에서 일어나기 힘들 것 같다,2,2,3,2,8,8,7,5,3,6,6,2,1,1,49,4,건축설비공학,1,1,1,40,5700,2700,NaN,140201.0


In [ ]:
bq1_list = total_2020.bq1.unique()
bq1_list

array([16, 15,  3,  6, 11, 19, 18, 17, 14,  7,  1,  4, 13, 10,  8,  9, 20,
        5, 12,  2, 21])

In [ ]:
for bq1 in bq1_list:
    mean_bq30_1 = round(total_2020.loc[lambda x: (x.bq1 == bq1) & (x.bq30_1.notnull())].bq30_1.apply(lambda x: int(x)).mean())
    train_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_1.isnull()), 'bq30_1'] = mean_bq30_1
    test_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_1.isnull()), 'bq30_1'] = mean_bq30_1
    
    mean_bq30_2 = round(total_2020.loc[lambda x: (x.bq1 == bq1) & (x.bq30_2.notnull())].bq30_2.apply(lambda x: int(x)).mean())
    train_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_2.isnull()), 'bq30_2'] = mean_bq30_2
    test_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_2.isnull()), 'bq30_2'] = mean_bq30_2
    
    try:
        mean_bq30_3 = round(total_2020.loc[lambda x: (x.bq1 == bq1) & (x.bq30_3.notnull())].bq30_3.apply(lambda x: int(x)).mean())
    except:
        mean_bq30_3 = mean_bq30_2
        
    train_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_3.isnull()), 'bq30_3'] = mean_bq30_3
    test_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_3.isnull()), 'bq30_3'] = mean_bq30_3

In [ ]:
for index, value, tindex, tvalue in zip(train_2020.isnull().sum().index, train_2020.isnull().sum(), test_2020.isnull().sum().index, test_2020.isnull().sum()):
    if (value + tvalue) != 0:
        print(f"{index} : {value} / {tindex} : {tvalue} / {value + tvalue}")

In [ ]:
%cd '/content/drive/MyDrive/직업추천/data'

/content/drive/MyDrive/직업추천/data


In [ ]:
train_2020.to_csv('2020_train_preprocessed.csv', index=False)
test_2020.to_csv('2020_test_preprocessed.csv', index=False)

In [ ]:
%cd '/content/drive/MyDrive/KNOW/KNOW_preprocessing'

train_2020.to_csv('train_2020.csv', index=False)
test_2020.to_csv('test_2020.csv', index=False)

/content/drive/MyDrive/KNOW/KNOW_preprocessing


In [ ]:
train_2020

,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,saq4_2,saq5_1,saq5_2,saq6_1,saq6_2,saq7_1,saq7_2,saq8_1,saq8_2,saq9_1,saq9_2,saq10_1,saq10_2,saq11_1,saq11_2,saq12_1,saq12_2,saq13_1,saq13_2,saq14_1,saq14_2,saq15_1,saq15_2,saq16_1,saq16_2,saq17_1,saq17_2,saq18_1,saq18_2,saq19_1,saq19_2,saq20_1,...,bq16_9,bq16_10,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq21_4,bq22_1,bq22_2,bq22_3,bq22_4,bq22_5,bq22_6,bq23_1,bq23_2,bq23_3,bq24,bq25,bq26,bq26_1,bq27_1,bq27_2,bq28,bq29,bq30_1,bq30_2,bq30_3,knowcode
0,27127,4,4,4,4,4,5,4,5,3,3,4,5,1,0,4,5,3,4,1,0,3,4,3,4,4,5,3,4,4,5,3,4,3,4,1,0,1,0,4,...,3,4,4,2,2,2,2,2,2,2,2,2,1,3,일정하게 인원이 정해져 있으므로 일자리 변화는 없을 것 같아서,3,2,3,2,5,7,4,5,2,3,3,4,3,2,42,4,간호학,1,1,1,40,4000,2600,4506,304003
1,27128,5,6,5,6,4,5,4,5,4,5,4,5,5,6,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,3,4,4,...,3,4,3,2,2,2,2,2,2,2,2,2,1,2,경기 침체,3,3,3,2,3,4,3,5,6,6,3,6,5,1,45,4,기계공학,1,1,1,40,4000,3000,6875,221401
2,27129,3,4,3,4,3,4,3,5,3,5,3,5,3,5,4,5,4,5,4,5,3,4,3,4,3,4,4,5,4,4,4,5,3,4,4,5,4,5,3,...,4,4,3,2,2,1,1,2,2,2,2,2,2,2,전체적으로 경기 악화 때문,3,2,3,2,5,7,5,5,5,5,5,5,5,1,38,4,기계,1,1,1,48,4300,3000,4710,815001
3,27130,4,5,2,3,3,3,4,5,3,4,3,5,4,4,3,3,3,2,1,0,2,3,3,4,3,5,3,2,3,3,4,4,3,4,1,0,5,6,3,...,3,2,2,2,2,2,2,2,2,2,2,2,1,3,사람의 상태를 소통하며 확인해서 그에 맞는 각가의 처방을 내려줘야 하는 일이라 대체불가,3,3,3,2,6,4,7,3,6,7,7,6,4,2,25,4,보건학,1,1,1,40,3500,3300,4506,304003
4,27131,5,6,4,5,3,5,4,4,4,6,4,5,4,5,5,6,4,5,4,5,4,5,4,5,5,6,4,5,1,0,1,0,4,5,4,5,4,5,5,...,3,5,4,2,2,2,2,2,2,2,2,2,2,2,건설경기가 아직은 침체에서 일어나기 힘들 것 같다,2,2,3,2,8,8,7,5,3,6,6,2,1,1,49,4,건축설비공학,1,1,1,40,5700,2700,4905,140201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8117,35244,4,6,3,5,3,3,1,0,5,5,2,3,1,0,3,3,4,4,3,5,2,4,1,0,1,0,2,4,1,0,4,4,1,0,1,0,1,0,1,...,3,2,4,2,2,2,2,2,2,2,2,2,2,3,건설 경기의 어려움으로 다소 어려움,3,3,3,2,7,5,6,3,4,6,4,3,2,1,45,6,지구과학,1,1,1,40,5100,4000,6176,121105
8118,35245,3,4,4,5,2,4,4,6,1,0,2,4,5,5,3,4,4,4,2,3,3,3,3,4,2,3,4,5,3,5,3,4,4,5,4,6,3,4,4,...,2,4,4,2,2,2,2,2,2,2,2,2,1,4,광고 홍보 관련 시장이 켜져서,2,2,2,1,6,4,3,7,5,6,5,7,6,2,33,4,광고학,1,1,1,45,3100,2600,6614,26102
8119,35246,5,6,3,4,3,4,4,5,5,7,4,6,1,0,2,2,2,4,2,2,2,2,1,0,2,2,2,2,2,3,1,0,3,4,2,3,1,0,3,...,3,3,5,2,2,2,2,2,2,2,2,2,1,3,보험은 계속 필요 할 것이다,2,2,3,2,8,7,5,6,6,6,2,2,5,1,45,4,국문,1,1,1,40,8000,3000,6614,31301
8120,35247,5,6,5,5,4,5,3,3,3,4,4,5,3,4,5,5,5,6,4,5,3,5,3,4,2,3,3,4,2,3,2,3,1,0,1,0,3,3,3,...,2,5,5,2,2,2,2,2,2,2,2,2,2,4,인구수명이 길어지므로서 신약개발은 갈수록 많아질 것이라서,3,3,3,2,3,3,1,4,3,5,7,4,3,2,36,5,화학공학,1,1,1,40,5700,3200,4710,154105


# 2019년 모델

# Non_error 파일 불러오기

In [ ]:
train_2019 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/train/train_2019_서술형살림.csv')
test_2019 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/test/test_2019_서술형살림.csv')

In [ ]:
train_2019 = train_2019.drop(index=7758, axis=0)
train_2019 = train_2019.drop(index=7251, axis=0)
train_2019 = train_2019.drop(index=3361, axis=0)
train_2019 = train_2019.drop(index=3309, axis=0)
train_2019 = train_2019.drop(index=2687, axis=0)
train_2019 = train_2019.drop(index=2610, axis=0)
train_2019 = train_2019.drop(index=1394, axis=0)
train_2019 = train_2019.drop(index=1302, axis=0)
train_2019 = train_2019.drop(index=614, axis=0)
train_2019 = train_2019.drop(index=177, axis=0)

In [ ]:
 train_2019.reset_index()

,index,Unnamed: 0,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,kq6_1,kq6_2,kq7_1,kq7_2,kq8_1,kq8_2,kq9_1,kq9_2,kq10_1,kq10_2,kq11_1,...,bq14_5,bq15,bq16_1,bq16_2,bq16_3,bq16_4,bq16_5,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq22,bq23,bq24,bq25,bq26,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
0,0,0,18569,4,4,4,3,4,4,4,4,4,4,4,5,4,4,4,4,3,3,2,2,1,0,2,2,2,2,3,4,4,5,1,0,4,5,4,5,4,...,5.0,4.0,4.0,4.0,4.0,3.0,4.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,사람은 줄고 일은 많으니까 업무가 많다,3.0,4,수송수요가 늘어날다,4.0,4.0,3,없음,없음,없음,1,35.0,4,기계공학과,1.0,1,0,1,40.0,3500,3000,5500,812301
1,1,1,18570,4,3,4,4,4,4,3,4,5,4,3,5,4,4,1,2,4,6,3,4,4,5,5,5,3,4,5,6,1,0,1,0,1,0,1,0,1,...,3.0,3.0,4.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,없음,1.0,3,운송시장이 큰 변화 없을듯해서,2.0,3.0,3,"컨테이너 대여사업,랜탈사업",없음,없음,1,63.0,3,경영학,2.0,0,5,0,40.0,4843,3128,5000,15201
2,2,2,18571,2,3,2,2,2,2,2,2,3,3,3,2,2,2,2,2,2,2,1,0,3,4,2,3,2,2,1,0,1,0,3,3,1,0,1,0,1,...,4.0,1.0,3.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,없음,1.0,3,없음,4.0,4.0,4,숙박업,없음,없음,2,62.0,2,없음,2.0,0,6,0,30.0,3293,2104,3000,901101
3,3,3,18572,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,7,5,6,5,6,5,6,4,5,4,5,4,5,5,6,3,3,3,...,4.0,3.0,4.0,3.0,3.0,3.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,없음,2.0,4,편리하게 일처리을 하기위해서 문서대행 업무를 맡길거 같아서,2.0,3.0,3,없음,없음,없음,2,33.0,4,행정학,1.0,1,0,1,40.0,3600,2400,5500,29903
4,4,4,18573,1,4,4,1,1,2,4,3,4,4,4,5,4,3,1,1,2,2,2,2,2,2,2,2,4,5,1,0,1,0,1,0,1,0,1,0,1,...,3.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,없음,2.0,2,일거리가 줄어들고 있다,3.0,2.0,2,용역회사(청소원),없음,없음,1,67.0,1,없음,2.0,0,6,0,45.0,3143,2254,1500,561401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8540,8550,8550,27122,3,4,4,3,4,3,3,3,3,3,4,3,3,4,4,4,2,2,2,2,2,3,2,3,2,3,2,3,1,0,1,0,3,4,2,2,3,...,5.0,4.0,2.0,3.0,3.0,2.0,2.0,3.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,방송장비의 기술변화로 장비도입 및 교육실시,1.0,2,과학 및 기계의 발달(1인 미디어),1.0,1.0,1,없음,없음,유튜버,1,41.0,4,방송영상과,1.0,1,0,1,52.0,4500,3000,4173,416105
8541,8551,8551,27123,5,5,4,4,4,5,4,4,5,5,4,4,3,2,3,4,1,0,1,0,1,0,4,6,5,6,1,0,1,0,1,0,1,0,1,0,1,...,4.0,4.0,3.0,3.0,4.0,2.0,3.0,5.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,없음,1.0,4,바문화형성,3.0,3.0,3,없음,소믈리에,없음,1,28.0,3,호텔관광학,1.0,1,0,1,60.0,3000,2400,4534,531601
8542,8552,8552,27124,3,3,4,4,4,4,4,3,3,4,3,3,2,4,4,2,2,2,3,3,3,4,4,5,4,4,1,0,4,4,2,2,3,3,1,0,2,...,3.0,4.0,4.0,4.0,4.0,3.0,2.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,없음,4.0,3,없음,3.0,3.0,2,총무사무원,없음,없음,2,50.0,2,상업과,1.0,3,0,1,40.0,2400,1600,3302,615301
8543,8553,8553,27125,3,3,3,1,4,4,3,2,3,3,4,4,2,3,2,4,3,3,2,2,3,3,1,0,2,3,2,4,3,3,2,3,3,4,4,4,3,...,3.0,3.0,3.0,3.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,없음,2.0,2,자동화,3.0,3.0,3,없음,농기계 수리 자영업,없음,1,48.0,3,기계,1.0,1,0,1,48.0,4500,2800,3096,811901


In [ ]:
test_2019.iloc[7090,:] = test_2019.iloc[7089,:]
test_2019.iloc[5933,:] = test_2019.iloc[5932,:]
test_2019.iloc[3694,:] = test_2019.iloc[3693,:]
test_2019.iloc[3197,:] = test_2019.iloc[3196,:]

# 중요도+수준

In [ ]:
train_2019["kq1"] = train_2019["kq1_1"] + train_2019["kq1_2"]
train_2019 = train_2019.drop(columns=["kq1_1","kq1_2"], axis=1)

train_2019["kq2"] = train_2019["kq2_1"] + train_2019["kq2_2"]
train_2019 = train_2019.drop(columns=["kq2_1","kq2_2"], axis=1)

train_2019["kq3"] = train_2019["kq3_1"] + train_2019["kq3_2"]
train_2019 = train_2019.drop(columns=["kq3_1","kq3_2"], axis=1)

train_2019["kq4"] = train_2019["kq4_1"] + train_2019["kq4_2"]
train_2019 = train_2019.drop(columns=["kq4_1","kq4_2"], axis=1)

train_2019["kq5"] = train_2019["kq5_1"] + train_2019["kq5_2"]
train_2019 = train_2019.drop(columns=["kq5_1","kq5_2"], axis=1)

train_2019["kq6"] = train_2019["kq6_1"] + train_2019["kq6_2"]
train_2019 = train_2019.drop(columns=["kq6_1","kq6_2"], axis=1)

train_2019["kq7"] = train_2019["kq7_1"] + train_2019["kq7_2"]
train_2019 = train_2019.drop(columns=["kq7_1","kq7_2"], axis=1)

train_2019["kq8"] = train_2019["kq8_1"] + train_2019["kq8_2"]
train_2019 = train_2019.drop(columns=["kq8_1","kq8_2"], axis=1)

train_2019["kq9"] = train_2019["kq9_1"] + train_2019["kq9_2"]
train_2019 = train_2019.drop(columns=["kq9_1","kq9_2"], axis=1)

train_2019["kq10"] = train_2019["kq10_1"] + train_2019["kq10_2"]
train_2019 = train_2019.drop(columns=["kq10_1","kq10_2"], axis=1)

train_2019["kq11"] = train_2019["kq11_1"] + train_2019["kq11_2"]
train_2019 = train_2019.drop(columns=["kq11_1","kq11_2"], axis=1)

train_2019["kq12"] = train_2019["kq12_1"] + train_2019["kq12_2"]
train_2019 = train_2019.drop(columns=["kq12_1","kq12_2"], axis=1)

train_2019["kq13"] = train_2019["kq13_1"] + train_2019["kq13_2"]
train_2019 = train_2019.drop(columns=["kq13_1","kq13_2"], axis=1)

train_2019["kq14"] = train_2019["kq14_1"] + train_2019["kq14_2"]
train_2019 = train_2019.drop(columns=["kq14_1","kq14_2"], axis=1)

train_2019["kq15"] = train_2019["kq15_1"] + train_2019["kq15_2"]
train_2019 = train_2019.drop(columns=["kq15_1","kq15_2"], axis=1)

train_2019["kq16"] = train_2019["kq16_1"] + train_2019["kq16_2"]
train_2019 = train_2019.drop(columns=["kq16_1","kq16_2"], axis=1)

train_2019["kq17"] = train_2019["kq17_1"] + train_2019["kq17_2"]
train_2019 = train_2019.drop(columns=["kq17_1","kq17_2"], axis=1)

train_2019["kq18"] = train_2019["kq18_1"] + train_2019["kq18_2"]
train_2019 = train_2019.drop(columns=["kq18_1","kq18_2"], axis=1)

train_2019["kq19"] = train_2019["kq19_1"] + train_2019["kq19_2"]
train_2019 = train_2019.drop(columns=["kq19_1","kq19_2"], axis=1)

train_2019["kq20"] = train_2019["kq20_1"] + train_2019["kq20_2"]
train_2019 = train_2019.drop(columns=["kq20_1","kq20_2"], axis=1)

train_2019["kq21"] = train_2019["kq21_1"] + train_2019["kq21_2"]
train_2019 = train_2019.drop(columns=["kq21_1","kq21_2"], axis=1)

train_2019["kq22"] = train_2019["kq22_1"] + train_2019["kq22_2"]
train_2019 = train_2019.drop(columns=["kq22_1","kq22_2"], axis=1)

train_2019["kq23"] = train_2019["kq23_1"] + train_2019["kq23_2"]
train_2019 = train_2019.drop(columns=["kq23_1","kq23_2"], axis=1)

train_2019["kq24"] = train_2019["kq24_1"] + train_2019["kq24_2"]
train_2019 = train_2019.drop(columns=["kq24_1","kq24_2"], axis=1)

train_2019["kq25"] = train_2019["kq25_1"] + train_2019["kq25_2"]
train_2019 = train_2019.drop(columns=["kq25_1","kq25_2"], axis=1)

train_2019["kq26"] = train_2019["kq26_1"] + train_2019["kq26_2"]
train_2019 = train_2019.drop(columns=["kq26_1","kq26_2"], axis=1)

train_2019["kq27"] = train_2019["kq27_1"] + train_2019["kq27_2"]
train_2019 = train_2019.drop(columns=["kq27_1","kq27_2"], axis=1)

train_2019["kq28"] = train_2019["kq28_1"] + train_2019["kq28_2"]
train_2019 = train_2019.drop(columns=["kq28_1","kq28_2"], axis=1)

train_2019["kq29"] = train_2019["kq29_1"] + train_2019["kq29_2"]
train_2019 = train_2019.drop(columns=["kq29_1","kq29_2"], axis=1)

train_2019["kq30"] = train_2019["kq30_1"] + train_2019["kq30_2"]
train_2019 = train_2019.drop(columns=["kq30_1","kq30_2"], axis=1)

train_2019["kq31"] = train_2019["kq31_1"] + train_2019["kq31_2"]
train_2019 = train_2019.drop(columns=["kq31_1","kq31_2"], axis=1)

train_2019["kq32"] = train_2019["kq32_1"] + train_2019["kq32_2"]
train_2019 = train_2019.drop(columns=["kq32_1","kq32_2"], axis=1)

train_2019["kq33"] = train_2019["kq33_1"] + train_2019["kq33_2"]
train_2019 = train_2019.drop(columns=["kq33_1","kq33_2"], axis=1)

In [ ]:
test_2019["kq1"] = test_2019["kq1_1"] + test_2019["kq1_2"]
test_2019 = test_2019.drop(columns=["kq1_1","kq1_2"], axis=1)

test_2019["kq2"] = test_2019["kq2_1"] + test_2019["kq2_2"]
test_2019 = test_2019.drop(columns=["kq2_1","kq2_2"], axis=1)

test_2019["kq3"] = test_2019["kq3_1"] + test_2019["kq3_2"]
test_2019 = test_2019.drop(columns=["kq3_1","kq3_2"], axis=1)

test_2019["kq4"] = test_2019["kq4_1"] + test_2019["kq4_2"]
test_2019 = test_2019.drop(columns=["kq4_1","kq4_2"], axis=1)

test_2019["kq5"] = test_2019["kq5_1"] + test_2019["kq5_2"]
test_2019 = test_2019.drop(columns=["kq5_1","kq5_2"], axis=1)

test_2019["kq6"] = test_2019["kq6_1"] + test_2019["kq6_2"]
test_2019 = test_2019.drop(columns=["kq6_1","kq6_2"], axis=1)

test_2019["kq7"] = test_2019["kq7_1"] + test_2019["kq7_2"]
test_2019 = test_2019.drop(columns=["kq7_1","kq7_2"], axis=1)

test_2019["kq8"] = test_2019["kq8_1"] + test_2019["kq8_2"]
test_2019 = test_2019.drop(columns=["kq8_1","kq8_2"], axis=1)

test_2019["kq9"] = test_2019["kq9_1"] + test_2019["kq9_2"]
test_2019 = test_2019.drop(columns=["kq9_1","kq9_2"], axis=1)

test_2019["kq10"] = test_2019["kq10_1"] + test_2019["kq10_2"]
test_2019 = test_2019.drop(columns=["kq10_1","kq10_2"], axis=1)

test_2019["kq11"] = test_2019["kq11_1"] + test_2019["kq11_2"]
test_2019 = test_2019.drop(columns=["kq11_1","kq11_2"], axis=1)

test_2019["kq12"] = test_2019["kq12_1"] + test_2019["kq12_2"]
test_2019 = test_2019.drop(columns=["kq12_1","kq12_2"], axis=1)

test_2019["kq13"] = test_2019["kq13_1"] + test_2019["kq13_2"]
test_2019 = test_2019.drop(columns=["kq13_1","kq13_2"], axis=1)

test_2019["kq14"] = test_2019["kq14_1"] + test_2019["kq14_2"]
test_2019 = test_2019.drop(columns=["kq14_1","kq14_2"], axis=1)

test_2019["kq15"] = test_2019["kq15_1"] + test_2019["kq15_2"]
test_2019 = test_2019.drop(columns=["kq15_1","kq15_2"], axis=1)

test_2019["kq16"] = test_2019["kq16_1"] + test_2019["kq16_2"]
test_2019 = test_2019.drop(columns=["kq16_1","kq16_2"], axis=1)

test_2019["kq17"] = test_2019["kq17_1"] + test_2019["kq17_2"]
test_2019 = test_2019.drop(columns=["kq17_1","kq17_2"], axis=1)

test_2019["kq18"] = test_2019["kq18_1"] + test_2019["kq18_2"]
test_2019 = test_2019.drop(columns=["kq18_1","kq18_2"], axis=1)

test_2019["kq19"] = test_2019["kq19_1"] + test_2019["kq19_2"]
test_2019 = test_2019.drop(columns=["kq19_1","kq19_2"], axis=1)

test_2019["kq20"] = test_2019["kq20_1"] + test_2019["kq20_2"]
test_2019 = test_2019.drop(columns=["kq20_1","kq20_2"], axis=1)

test_2019["kq21"] = test_2019["kq21_1"] + test_2019["kq21_2"]
test_2019 = test_2019.drop(columns=["kq21_1","kq21_2"], axis=1)

test_2019["kq22"] = test_2019["kq22_1"] + test_2019["kq22_2"]
test_2019 = test_2019.drop(columns=["kq22_1","kq22_2"], axis=1)

test_2019["kq23"] = test_2019["kq23_1"] + test_2019["kq23_2"]
test_2019 = test_2019.drop(columns=["kq23_1","kq23_2"], axis=1)

test_2019["kq24"] = test_2019["kq24_1"] + test_2019["kq24_2"]
test_2019 = test_2019.drop(columns=["kq24_1","kq24_2"], axis=1)

test_2019["kq25"] = test_2019["kq25_1"] + test_2019["kq25_2"]
test_2019 = test_2019.drop(columns=["kq25_1","kq25_2"], axis=1)

test_2019["kq26"] = test_2019["kq26_1"] + test_2019["kq26_2"]
test_2019 = test_2019.drop(columns=["kq26_1","kq26_2"], axis=1)

test_2019["kq27"] = test_2019["kq27_1"] + test_2019["kq27_2"]
test_2019 = test_2019.drop(columns=["kq27_1","kq27_2"], axis=1)

test_2019["kq28"] = test_2019["kq28_1"] + test_2019["kq28_2"]
test_2019 = test_2019.drop(columns=["kq28_1","kq28_2"], axis=1)

test_2019["kq29"] = test_2019["kq29_1"] + test_2019["kq29_2"]
test_2019 = test_2019.drop(columns=["kq29_1","kq29_2"], axis=1)

test_2019["kq30"] = test_2019["kq30_1"] + test_2019["kq30_2"]
test_2019 = test_2019.drop(columns=["kq30_1","kq30_2"], axis=1)

test_2019["kq31"] = test_2019["kq31_1"] + test_2019["kq31_2"]
test_2019 = test_2019.drop(columns=["kq31_1","kq31_2"], axis=1)

test_2019["kq32"] = test_2019["kq32_1"] + test_2019["kq32_2"]
test_2019 = test_2019.drop(columns=["kq32_1","kq32_2"], axis=1)

test_2019["kq33"] = test_2019["kq33_1"] + test_2019["kq33_2"]
test_2019 = test_2019.drop(columns=["kq33_1","kq33_2"], axis=1)

# 처리

In [ ]:
# 문자열 공백 제거
train_2019['bq4_1a']=train_2019['bq4_1a'].str.replace(" ", "")
train_2019['bq4_1b']=train_2019['bq4_1b'].str.replace(" ", "")
train_2019['bq4_1c']=train_2019['bq4_1c'].str.replace(" ", "")
train_2019['bq5_2']=train_2019['bq5_2'].str.replace(" ", "")
train_2019['bq18_10']=train_2019['bq18_10'].str.replace(" ", "")
train_2019['bq20_1']=train_2019['bq20_1'].str.replace(" ", "")
train_2019['bq22']=train_2019['bq22'].str.replace(" ", "")
train_2019['bq23']=train_2019['bq23'].str.replace(" ", "")
train_2019['bq24']=train_2019['bq24'].str.replace(" ", "")
train_2019['bq27_1']=train_2019['bq27_1'].str.replace(" ", "")

# 영어 대문자로 통일
train_2019['bq4_1a']=train_2019['bq4_1a'].str.upper()
train_2019['bq4_1b']=train_2019['bq4_1b'].str.upper()
train_2019['bq4_1c']=train_2019['bq4_1c'].str.upper()
train_2019['bq5_2']=train_2019['bq5_2'].str.upper()
train_2019['bq18_10']=train_2019['bq18_10'].str.upper()
train_2019['bq20_1']=train_2019['bq20_1'].str.upper()
train_2019['bq22']=train_2019['bq22'].str.upper()
train_2019['bq23']=train_2019['bq23'].str.upper()
train_2019['bq24']=train_2019['bq24'].str.upper()
train_2019['bq27_1']=train_2019['bq27_1'].str.upper()

In [ ]:
# 문자열 공백 제거
test_2019['bq4_1a']=test_2019['bq4_1a'].str.replace(" ", "")
test_2019['bq4_1b']=test_2019['bq4_1b'].str.replace(" ", "")
test_2019['bq4_1c']=test_2019['bq4_1c'].str.replace(" ", "")
test_2019['bq5_2']=test_2019['bq5_2'].str.replace(" ", "")
test_2019['bq18_10']=test_2019['bq18_10'].str.replace(" ", "")
test_2019['bq20_1']=test_2019['bq20_1'].str.replace(" ", "")
test_2019['bq22']=test_2019['bq22'].str.replace(" ", "")
test_2019['bq23']=test_2019['bq23'].str.replace(" ", "")
test_2019['bq24']=test_2019['bq24'].str.replace(" ", "")
test_2019['bq27_1']=test_2019['bq27_1'].str.replace(" ", "")

# 영어 대문자로 통일
test_2019['bq4_1a']=test_2019['bq4_1a'].str.upper()
test_2019['bq4_1b']=test_2019['bq4_1b'].str.upper()
test_2019['bq4_1c']=test_2019['bq4_1c'].str.upper()
test_2019['bq5_2']=test_2019['bq5_2'].str.upper()
test_2019['bq18_10']=test_2019['bq18_10'].str.upper()
test_2019['bq20_1']=test_2019['bq20_1'].str.upper()
test_2019['bq22']=test_2019['bq22'].str.upper()
test_2019['bq23']=test_2019['bq23'].str.upper()
test_2019['bq24']=test_2019['bq24'].str.upper()
test_2019['bq27_1']=test_2019['bq27_1'].str.upper()

In [ ]:
for j in range(8545):
    if train_2019.bq27_1.iloc[j] == '공':
      train_2019.bq27_1.iloc[j] = '공고공대'
for j in range(8545):
    if train_2019.bq27_1.iloc[j] == '농':
      train_2019.bq27_1.iloc[j] = '농업'
for j in range(8545):
    if train_2019.bq27_1.iloc[j] == '문':
      train_2019.bq27_1.iloc[j] = '문과'
for j in range(8545):
    if train_2019.bq27_1.iloc[j] == '외':
      train_2019.bq27_1.iloc[j] = '외국어'
for j in range(8545):
    if train_2019.bq27_1.iloc[j] == '상':
      train_2019.bq27_1.iloc[j] = '상고상업상경'
for j in range(8545):
    if train_2019.bq27_1.iloc[j] == '약':
      train_2019.bq27_1.iloc[j] = '약학약사'
for j in range(8545):
    if train_2019.bq27_1.iloc[j] == '의':
      train_2019.bq27_1.iloc[j] = '의사전문의'
for j in range(8545):
    if train_2019.bq27_1.iloc[j] == '이':
      train_2019.bq27_1.iloc[j] = '이과'

In [ ]:
for j in range(8554):
    if test_2019.bq27_1.iloc[j] == '공':
      test_2019.bq27_1.iloc[j] = '공고공대'
for j in range(8554):
    if test_2019.bq27_1.iloc[j] == '농':
      test_2019.bq27_1.iloc[j] = '농업'
for j in range(8554):
    if test_2019.bq27_1.iloc[j] == '문':
      test_2019.bq27_1.iloc[j] = '문과'
for j in range(8554):
    if test_2019.bq27_1.iloc[j] == '외':
      test_2019.bq27_1.iloc[j] = '외국어'
for j in range(8554):
    if test_2019.bq27_1.iloc[j] == '상':
      test_2019.bq27_1.iloc[j] = '상고상업상경'
for j in range(8554):
    if test_2019.bq27_1.iloc[j] == '약':
      test_2019.bq27_1.iloc[j] = '약학약사'
for j in range(8554):
    if test_2019.bq27_1.iloc[j] == '의':
      test_2019.bq27_1.iloc[j] = '의사전문의'
for j in range(8554):
    if test_2019.bq27_1.iloc[j] == '이':
      test_2019.bq27_1.iloc[j] = '이과'

In [ ]:
testmajorfix = pd.read_csv('/content/gdrive/MyDrive/테스트학과수정용4.csv')

for i in range(235):
  train_2019.bq27_1 = train_2019.bq27_1.str.replace(testmajorfix.수정전.iloc[i], testmajorfix.수정후.iloc[i])

for i in range(235):
  test_2019.bq27_1 = test_2019.bq27_1.str.replace(testmajorfix.수정전.iloc[i], testmajorfix.수정후.iloc[i])

In [ ]:
train_2019.bq27_1 = train_2019.bq27_1.str.replace("-", "")
test_2019.bq27_1 = test_2019.bq27_1.str.replace("-", "")

In [ ]:
'bq4_1a','bq4_1b','bq4_1c','bq5_2','bq18_10','bq20_1','bq22','bq23','bq24','bq27_1'

('bq4_1a',
 'bq4_1b',
 'bq4_1c',
 'bq5_2',
 'bq18_10',
 'bq20_1',
 'bq22',
 'bq23',
 'bq24',
 'bq27_1')

In [ ]:
train_2019["key_word"] = train_2019["bq4_1a"] + " " + train_2019["bq4_1b"] + " " + train_2019["bq4_1c"] + " " + train_2019["bq5_2"] + " " + train_2019["bq18_10"] + " " + train_2019["bq20_1"] + " " + train_2019["bq22"] + " " + train_2019["bq23"] + " " + train_2019["bq24"] + " " + train_2019["bq27_1"]

In [ ]:
test_2019["key_word"] = test_2019["bq4_1a"] + " " + test_2019["bq4_1b"] + " " + test_2019["bq4_1c"] + " " + test_2019["bq5_2"] + " " + test_2019["bq18_10"] + " " + test_2019["bq20_1"] + " " + test_2019["bq22"] + " " + test_2019["bq23"] + " " + test_2019["bq24"] + " " + test_2019["bq27_1"]

In [ ]:
train_2019 = train_2019.drop(columns=['bq4_1a','bq4_1b','bq4_1c','bq5_2','bq18_10','bq20_1','bq22','bq23','bq24','bq27_1'], axis=1)

In [ ]:
test_2019 =test_2019.drop(columns=['bq4_1a','bq4_1b','bq4_1c','bq5_2','bq18_10','bq20_1','bq22','bq23','bq24','bq27_1'], axis=1)

In [ ]:
keyword = pd.read_csv('/content/gdrive/MyDrive/키워드카운트수정6.csv')
keyword

,word
0,3D
1,AFP
2,AKLU
3,AP
4,ASNT
...,...
888,회복
889,회사
890,회전익
891,회화


In [ ]:
k_list = []
for i in range(893):
  k_list.append(str(keyword.iloc[i,0]))

In [ ]:
plus_col = []
for w in range(len(k_list)):
    plus_col.append([0 for i in range(len(train_2019))])
    
for idx in tqdm(range(len(train_2019))):
    temp_str = str(train_2019.key_word.iloc[idx])
    for i in range(len(k_list)):
        plus_col[i][idx] = temp_str.count(k_list[i])
        
for i in range(len(plus_col)):
    train_2019[k_list[i]] = plus_col[i]
train_2019

100%|██████████| 8545/8545 [00:04<00:00, 2119.91it/s]


,Unnamed: 0,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,bq1,bq2,bq3,bq4,bq5,bq5_1,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13_1,bq13_2,bq13_3,...,항해,항해사,해기사,해설,해양,해외,행정,헤드셋,헤어,헬기,헬리,헬멧,현미경,현장,혈압,협회,호루라기,호스피스,호텔,호흡기,홍보,화공,화물,화약,화장품,화재,화학,화학공,화훼,환경,환경공,환자,활용,회계,회계사,회복,회사,회전익,회화,훈련
0,0,18569,4,4,4,3,4,4,4,4,4,4,4,5,4,4,4,4,15,2,4,1,1,5,7,4,4,4,3,4,4,4,5,5,4,4,4,4.0,3,4.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,18570,4,3,4,4,4,4,3,4,5,4,3,5,4,4,1,2,8,1,3,2,2,0,1,4,4,3,3,4,4,3,4,9,9,4,4,1.0,4,3.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,18571,2,3,2,2,2,2,2,2,3,3,3,2,2,2,2,2,1,4,1,2,2,0,2,2,4,4,4,4,4,4,3,9,9,9,3,4.0,4,4.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,18572,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,15,1,2,1,1,1,4,5,5,5,5,4,4,4,4,3,3,4,4,5.0,5,5.0,...,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,4,18573,1,4,4,1,1,2,4,3,4,4,4,5,4,3,1,1,19,4,1,2,2,0,7,1,2,2,2,4,2,3,2,9,9,9,2,3.0,2,2.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8550,8550,27122,3,4,4,3,4,3,3,3,3,3,4,3,3,4,4,4,10,3,4,1,1,1,4,2,4,2,3,3,4,1,1,3,4,4,4,3.0,3,3.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8551,8551,27123,5,5,4,4,4,5,4,4,5,5,4,4,3,2,3,4,9,1,1,1,1,3,4,2,3,3,3,3,3,2,3,3,5,5,5,4.0,3,2.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8552,8552,27124,3,3,4,4,4,4,4,3,3,4,3,3,2,4,4,2,7,1,1,2,2,0,5,4,3,3,2,2,4,3,3,3,4,3,3,2.0,3,3.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8553,8553,27125,3,3,3,1,4,4,3,2,3,3,4,4,2,3,2,4,19,4,2,1,2,0,3,2,3,3,2,3,3,2,3,3,3,3,3,3.0,3,3.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
plus_col = []
for w in range(len(k_list)):
    plus_col.append([0 for i in range(len(test_2019))])
    
for idx in tqdm(range(len(test_2019))):
    temp_str = str(test_2019.key_word.iloc[idx])
    for i in range(len(k_list)):
        plus_col[i][idx] = temp_str.count(k_list[i])
        
for i in range(len(plus_col)):
    test_2019[k_list[i]] = plus_col[i]
test_2019

100%|██████████| 8554/8554 [00:04<00:00, 2134.27it/s]


,Unnamed: 0,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,bq1,bq2,bq3,bq4,bq5,bq5_1,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13_1,bq13_2,bq13_3,...,항해,항해사,해기사,해설,해양,해외,행정,헤드셋,헤어,헬기,헬리,헬멧,현미경,현장,혈압,협회,호루라기,호스피스,호텔,호흡기,홍보,화공,화물,화약,화장품,화재,화학,화학공,화훼,환경,환경공,환자,활용,회계,회계사,회복,회사,회전익,회화,훈련
0,0,18569,5,5,5,4,4,5,4,5,5,4,5,4,4,4,3,4,15,2,4,1,2,0,5,4,3,3,3,3,3,3,3,3,3,3,3,2.0,3,3.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,18570,2,3,3,1,3,2,3,3,3,2,3,4,3,2,1,3,3,1,2,2,2,0,7,2,3,1,2,2,3,2,3,3,4,4,3,4.0,2,2.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,18571,5,4,5,4,4,4,4,5,4,4,4,5,4,4,4,4,6,1,2,1,2,0,7,4,3,3,3,4,3,3,3,4,3,4,4,3.0,4,4.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,3,18572,3,3,3,2,3,3,3,3,2,2,3,3,3,3,2,2,8,1,2,1,1,2,1,2,3,2,1,4,4,3,2,9,9,2,2,3.0,4,3.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,18573,5,4,5,3,3,3,3,4,5,4,5,4,5,3,5,4,13,2,5,2,1,1,6,7,4,4,4,4,4,4,3,4,5,4,4,2.0,4,4.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8549,8549,27122,3,4,5,2,4,4,4,5,5,4,5,3,4,3,1,1,14,1,2,1,1,3,2,2,3,2,3,2,3,3,1,2,3,3,3,4.0,3,3.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
8550,8550,27123,4,4,3,4,3,4,4,3,3,4,4,5,4,3,4,4,18,1,1,2,2,0,7,4,4,4,5,4,4,4,3,3,4,4,4,2.0,4,4.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8551,8551,27124,3,4,4,2,2,2,3,4,4,3,4,4,4,2,2,1,15,2,6,2,1,4,7,1,4,2,2,3,3,3,4,3,4,4,4,2.0,3,3.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8552,8552,27125,4,4,3,3,4,3,4,3,4,3,3,4,3,4,3,2,18,1,2,2,2,0,7,5,4,4,3,4,3,4,3,4,5,4,4,1.0,4,3.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_2019 = train_2019.drop(columns=["Unnamed: 0","idx", "key_word"], axis=1)
test_2019 = test_2019.drop(columns=["Unnamed: 0","idx", "key_word"], axis=1)

In [ ]:
train_2019.to_csv('/content/gdrive/MyDrive/KNOW_data/train/train_2019_표석규.csv', index=False)
test_2019.to_csv('/content/gdrive/MyDrive/KNOW_data/test/test_2019_표석규.csv', index=False)

# 카테고리컬 처리

In [ ]:
train_2019 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/train/train_2019_표석규.csv')
test_2019 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/test/test_2019_표석규.csv')

In [ ]:
train_2019.bq1

0       15
1        3
2        6
3        8
4       13
        ..
8539    14
8540    18
8541    15
8542    18
8543    18
Name: bq1, Length: 8544, dtype: int64

In [ ]:
train_2019['bq1'] = pd.Categorical(train_2019.bq1)
train_2019['bq2'] = pd.Categorical(train_2019.bq2)
train_2019['bq28_1'] = pd.Categorical(train_2019.bq28_1)
train_2019['bq28_2'] = pd.Categorical(train_2019.bq28_2)

test_2019['bq1'] = pd.Categorical(test_2019.bq1)
test_2019['bq2'] = pd.Categorical(test_2019.bq2)
test_2019['bq28_1'] = pd.Categorical(test_2019.bq28_1)
test_2019['bq28_2'] = pd.Categorical(test_2019.bq28_2)

In [ ]:
y = train_2019.knowcode

In [ ]:
train_2019 = train_2019.drop(columns=["knowcode"], axis=1)

In [ ]:
X = train_2019

In [ ]:
y = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/train/train_2019(Non_error).csv')
y = y.knowcode

# 모델

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

!pip install catboost
from catboost import CatBoostClassifier

     |████████████████████████████████| 76.1 MB 106 kB/s 


In [ ]:
#X = train_2019
#y = y_train_2019

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, stratify=y, random_state=777)

In [ ]:
md = RandomForestClassifier(n_estimators=3000, random_state=777)
md.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1200, random_state=777)

In [ ]:
y_train_hat = md.predict(X_train)
print(accuracy_score(y_train,y_train_hat))
y_valid_hat = md.predict(X_valid)
print(accuracy_score(y_valid, y_valid_hat))
print(f1_score(y_train,y_train_hat, average='macro'))
print(f1_score(y_valid, y_valid_hat, average='macro'))

1.0
0.6947368421052632
1.0
0.6408611997321675


In [ ]:
sample = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/sample_submission_수정7.csv')

In [ ]:
testpred = md.predict(test_2019)

for i in range(8554):
  sample.knowcode.iloc[i+18555] = testpred[i]

In [ ]:
sample.to_csv('/content/gdrive/MyDrive/KNOW_data/sample_submission_수정8.csv', index=False)

# 2020년 모델

# 중요도+수준

In [ ]:
train_2020 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/train/train_2020.csv')
test_2020 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/test/test_2020.csv')

In [ ]:
train_2020["saq1"] = train_2020["saq1_1"] + train_2020["saq1_2"]
train_2020 = train_2020.drop(columns=["saq1_1","saq1_2"], axis=1)

train_2020["saq2"] = train_2020["saq2_1"] + train_2020["saq2_2"]
train_2020 = train_2020.drop(columns=["saq2_1","saq2_2"], axis=1)

train_2020["saq3"] = train_2020["saq3_1"] + train_2020["saq3_2"]
train_2020 = train_2020.drop(columns=["saq3_1","saq3_2"], axis=1)

train_2020["saq4"] = train_2020["saq4_1"] + train_2020["saq4_2"]
train_2020 = train_2020.drop(columns=["saq4_1","saq4_2"], axis=1)

train_2020["saq5"] = train_2020["saq5_1"] + train_2020["saq5_2"]
train_2020 = train_2020.drop(columns=["saq5_1","saq5_2"], axis=1)

train_2020["saq6"] = train_2020["saq6_1"] + train_2020["saq6_2"]
train_2020 = train_2020.drop(columns=["saq6_1","saq6_2"], axis=1)

train_2020["saq7"] = train_2020["saq7_1"] + train_2020["saq7_2"]
train_2020 = train_2020.drop(columns=["saq7_1","saq7_2"], axis=1)

train_2020["saq8"] = train_2020["saq8_1"] + train_2020["saq8_2"]
train_2020 = train_2020.drop(columns=["saq8_1","saq8_2"], axis=1)

train_2020["saq9"] = train_2020["saq9_1"] + train_2020["saq9_2"]
train_2020 = train_2020.drop(columns=["saq9_1","saq9_2"], axis=1)

train_2020["saq10"] = train_2020["saq10_1"] + train_2020["saq10_2"]
train_2020 = train_2020.drop(columns=["saq10_1","saq10_2"], axis=1)

train_2020["saq11"] = train_2020["saq11_1"] + train_2020["saq11_2"]
train_2020 = train_2020.drop(columns=["saq11_1","saq11_2"], axis=1)

train_2020["saq12"] = train_2020["saq12_1"] + train_2020["saq12_2"]
train_2020 = train_2020.drop(columns=["saq12_1","saq12_2"], axis=1)

train_2020["saq13"] = train_2020["saq13_1"] + train_2020["saq13_2"]
train_2020 = train_2020.drop(columns=["saq13_1","saq13_2"], axis=1)

train_2020["saq14"] = train_2020["saq14_1"] + train_2020["saq14_2"]
train_2020 = train_2020.drop(columns=["saq14_1","saq14_2"], axis=1)

train_2020["saq15"] = train_2020["saq15_1"] + train_2020["saq15_2"]
train_2020 = train_2020.drop(columns=["saq15_1","saq15_2"], axis=1)

train_2020["saq16"] = train_2020["saq16_1"] + train_2020["saq16_2"]
train_2020 = train_2020.drop(columns=["saq16_1","saq16_2"], axis=1)

train_2020["saq17"] = train_2020["saq17_1"] + train_2020["saq17_2"]
train_2020 = train_2020.drop(columns=["saq17_1","saq17_2"], axis=1)

train_2020["saq18"] = train_2020["saq18_1"] + train_2020["saq18_2"]
train_2020 = train_2020.drop(columns=["saq18_1","saq18_2"], axis=1)

train_2020["saq19"] = train_2020["saq19_1"] + train_2020["saq19_2"]
train_2020 = train_2020.drop(columns=["saq19_1","saq19_2"], axis=1)

train_2020["saq20"] = train_2020["saq20_1"] + train_2020["saq20_2"]
train_2020 = train_2020.drop(columns=["saq20_1","saq20_2"], axis=1)

train_2020["saq21"] = train_2020["saq21_1"] + train_2020["saq21_2"]
train_2020 = train_2020.drop(columns=["saq21_1","saq21_2"], axis=1)

train_2020["saq22"] = train_2020["saq22_1"] + train_2020["saq22_2"]
train_2020 = train_2020.drop(columns=["saq22_1","saq22_2"], axis=1)

train_2020["saq23"] = train_2020["saq23_1"] + train_2020["saq23_2"]
train_2020 = train_2020.drop(columns=["saq23_1","saq23_2"], axis=1)

train_2020["saq24"] = train_2020["saq24_1"] + train_2020["saq24_2"]
train_2020 = train_2020.drop(columns=["saq24_1","saq24_2"], axis=1)

train_2020["saq25"] = train_2020["saq25_1"] + train_2020["saq25_2"]
train_2020 = train_2020.drop(columns=["saq25_1","saq25_2"], axis=1)

train_2020["saq26"] = train_2020["saq26_1"] + train_2020["saq26_2"]
train_2020 = train_2020.drop(columns=["saq26_1","saq26_2"], axis=1)

train_2020["saq27"] = train_2020["saq27_1"] + train_2020["saq27_2"]
train_2020 = train_2020.drop(columns=["saq27_1","saq27_2"], axis=1)

train_2020["saq28"] = train_2020["saq28_1"] + train_2020["saq28_2"]
train_2020 = train_2020.drop(columns=["saq28_1","saq28_2"], axis=1)

train_2020["saq29"] = train_2020["saq29_1"] + train_2020["saq29_2"]
train_2020 = train_2020.drop(columns=["saq29_1","saq29_2"], axis=1)

train_2020["saq30"] = train_2020["saq30_1"] + train_2020["saq30_2"]
train_2020 = train_2020.drop(columns=["saq30_1","saq30_2"], axis=1)

train_2020["saq31"] = train_2020["saq31_1"] + train_2020["saq31_2"]
train_2020 = train_2020.drop(columns=["saq31_1","saq31_2"], axis=1)

train_2020["saq32"] = train_2020["saq32_1"] + train_2020["saq32_2"]
train_2020 = train_2020.drop(columns=["saq32_1","saq32_2"], axis=1)

train_2020["saq33"] = train_2020["saq33_1"] + train_2020["saq33_2"]
train_2020 = train_2020.drop(columns=["saq33_1","saq33_2"], axis=1)

train_2020["saq34"] = train_2020["saq34_1"] + train_2020["saq34_2"]
train_2020 = train_2020.drop(columns=["saq34_1","saq34_2"], axis=1)

train_2020["saq35"] = train_2020["saq35_1"] + train_2020["saq35_2"]
train_2020 = train_2020.drop(columns=["saq35_1","saq35_2"], axis=1)

train_2020["saq36"] = train_2020["saq36_1"] + train_2020["saq36_2"]
train_2020 = train_2020.drop(columns=["saq36_1","saq36_2"], axis=1)

train_2020["saq37"] = train_2020["saq37_1"] + train_2020["saq37_2"]
train_2020 = train_2020.drop(columns=["saq37_1","saq37_2"], axis=1)

train_2020["saq38"] = train_2020["saq38_1"] + train_2020["saq38_2"]
train_2020 = train_2020.drop(columns=["saq38_1","saq38_2"], axis=1)

train_2020["saq39"] = train_2020["saq39_1"] + train_2020["saq39_2"]
train_2020 = train_2020.drop(columns=["saq39_1","saq39_2"], axis=1)

train_2020["saq40"] = train_2020["saq40_1"] + train_2020["saq40_2"]
train_2020 = train_2020.drop(columns=["saq40_1","saq40_2"], axis=1)

train_2020["saq41"] = train_2020["saq41_1"] + train_2020["saq41_2"]
train_2020 = train_2020.drop(columns=["saq41_1","saq41_2"], axis=1)

train_2020["saq42"] = train_2020["saq42_1"] + train_2020["saq42_2"]
train_2020 = train_2020.drop(columns=["saq42_1","saq42_2"], axis=1)

train_2020["saq43"] = train_2020["saq43_1"] + train_2020["saq43_2"]
train_2020 = train_2020.drop(columns=["saq43_1","saq43_2"], axis=1)

train_2020["saq44"] = train_2020["saq44_1"] + train_2020["saq44_2"]
train_2020 = train_2020.drop(columns=["saq44_1","saq44_2"], axis=1)

In [ ]:
test_2020["saq1"] = test_2020["saq1_1"] + test_2020["saq1_2"]
test_2020 = test_2020.drop(columns=["saq1_1","saq1_2"], axis=1)

test_2020["saq2"] = test_2020["saq2_1"] + test_2020["saq2_2"]
test_2020 = test_2020.drop(columns=["saq2_1","saq2_2"], axis=1)

test_2020["saq3"] = test_2020["saq3_1"] + test_2020["saq3_2"]
test_2020 = test_2020.drop(columns=["saq3_1","saq3_2"], axis=1)

test_2020["saq4"] = test_2020["saq4_1"] + test_2020["saq4_2"]
test_2020 = test_2020.drop(columns=["saq4_1","saq4_2"], axis=1)

test_2020["saq5"] = test_2020["saq5_1"] + test_2020["saq5_2"]
test_2020 = test_2020.drop(columns=["saq5_1","saq5_2"], axis=1)

test_2020["saq6"] = test_2020["saq6_1"] + test_2020["saq6_2"]
test_2020 = test_2020.drop(columns=["saq6_1","saq6_2"], axis=1)

test_2020["saq7"] = test_2020["saq7_1"] + test_2020["saq7_2"]
test_2020 = test_2020.drop(columns=["saq7_1","saq7_2"], axis=1)

test_2020["saq8"] = test_2020["saq8_1"] + test_2020["saq8_2"]
test_2020 = test_2020.drop(columns=["saq8_1","saq8_2"], axis=1)

test_2020["saq9"] = test_2020["saq9_1"] + test_2020["saq9_2"]
test_2020 = test_2020.drop(columns=["saq9_1","saq9_2"], axis=1)

test_2020["saq10"] = test_2020["saq10_1"] + test_2020["saq10_2"]
test_2020 = test_2020.drop(columns=["saq10_1","saq10_2"], axis=1)

test_2020["saq11"] = test_2020["saq11_1"] + test_2020["saq11_2"]
test_2020 = test_2020.drop(columns=["saq11_1","saq11_2"], axis=1)

test_2020["saq12"] = test_2020["saq12_1"] + test_2020["saq12_2"]
test_2020 = test_2020.drop(columns=["saq12_1","saq12_2"], axis=1)

test_2020["saq13"] = test_2020["saq13_1"] + test_2020["saq13_2"]
test_2020 = test_2020.drop(columns=["saq13_1","saq13_2"], axis=1)

test_2020["saq14"] = test_2020["saq14_1"] + test_2020["saq14_2"]
test_2020 = test_2020.drop(columns=["saq14_1","saq14_2"], axis=1)

test_2020["saq15"] = test_2020["saq15_1"] + test_2020["saq15_2"]
test_2020 = test_2020.drop(columns=["saq15_1","saq15_2"], axis=1)

test_2020["saq16"] = test_2020["saq16_1"] + test_2020["saq16_2"]
test_2020 = test_2020.drop(columns=["saq16_1","saq16_2"], axis=1)

test_2020["saq17"] = test_2020["saq17_1"] + test_2020["saq17_2"]
test_2020 = test_2020.drop(columns=["saq17_1","saq17_2"], axis=1)

test_2020["saq18"] = test_2020["saq18_1"] + test_2020["saq18_2"]
test_2020 = test_2020.drop(columns=["saq18_1","saq18_2"], axis=1)

test_2020["saq19"] = test_2020["saq19_1"] + test_2020["saq19_2"]
test_2020 = test_2020.drop(columns=["saq19_1","saq19_2"], axis=1)

test_2020["saq20"] = test_2020["saq20_1"] + test_2020["saq20_2"]
test_2020 = test_2020.drop(columns=["saq20_1","saq20_2"], axis=1)

test_2020["saq21"] = test_2020["saq21_1"] + test_2020["saq21_2"]
test_2020 = test_2020.drop(columns=["saq21_1","saq21_2"], axis=1)

test_2020["saq22"] = test_2020["saq22_1"] + test_2020["saq22_2"]
test_2020 = test_2020.drop(columns=["saq22_1","saq22_2"], axis=1)

test_2020["saq23"] = test_2020["saq23_1"] + test_2020["saq23_2"]
test_2020 = test_2020.drop(columns=["saq23_1","saq23_2"], axis=1)

test_2020["saq24"] = test_2020["saq24_1"] + test_2020["saq24_2"]
test_2020 = test_2020.drop(columns=["saq24_1","saq24_2"], axis=1)

test_2020["saq25"] = test_2020["saq25_1"] + test_2020["saq25_2"]
test_2020 = test_2020.drop(columns=["saq25_1","saq25_2"], axis=1)

test_2020["saq26"] = test_2020["saq26_1"] + test_2020["saq26_2"]
test_2020 = test_2020.drop(columns=["saq26_1","saq26_2"], axis=1)

test_2020["saq27"] = test_2020["saq27_1"] + test_2020["saq27_2"]
test_2020 = test_2020.drop(columns=["saq27_1","saq27_2"], axis=1)

test_2020["saq28"] = test_2020["saq28_1"] + test_2020["saq28_2"]
test_2020 = test_2020.drop(columns=["saq28_1","saq28_2"], axis=1)

test_2020["saq29"] = test_2020["saq29_1"] + test_2020["saq29_2"]
test_2020 = test_2020.drop(columns=["saq29_1","saq29_2"], axis=1)

test_2020["saq30"] = test_2020["saq30_1"] + test_2020["saq30_2"]
test_2020 = test_2020.drop(columns=["saq30_1","saq30_2"], axis=1)

test_2020["saq31"] = test_2020["saq31_1"] + test_2020["saq31_2"]
test_2020 = test_2020.drop(columns=["saq31_1","saq31_2"], axis=1)

test_2020["saq32"] = test_2020["saq32_1"] + test_2020["saq32_2"]
test_2020 = test_2020.drop(columns=["saq32_1","saq32_2"], axis=1)

test_2020["saq33"] = test_2020["saq33_1"] + test_2020["saq33_2"]
test_2020 = test_2020.drop(columns=["saq33_1","saq33_2"], axis=1)

test_2020["saq34"] = test_2020["saq34_1"] + test_2020["saq34_2"]
test_2020 = test_2020.drop(columns=["saq34_1","saq34_2"], axis=1)

test_2020["saq35"] = test_2020["saq35_1"] + test_2020["saq35_2"]
test_2020 = test_2020.drop(columns=["saq35_1","saq35_2"], axis=1)

test_2020["saq36"] = test_2020["saq36_1"] + test_2020["saq36_2"]
test_2020 = test_2020.drop(columns=["saq36_1","saq36_2"], axis=1)

test_2020["saq37"] = test_2020["saq37_1"] + test_2020["saq37_2"]
test_2020 = test_2020.drop(columns=["saq37_1","saq37_2"], axis=1)

test_2020["saq38"] = test_2020["saq38_1"] + test_2020["saq38_2"]
test_2020 = test_2020.drop(columns=["saq38_1","saq38_2"], axis=1)

test_2020["saq39"] = test_2020["saq39_1"] + test_2020["saq39_2"]
test_2020 = test_2020.drop(columns=["saq39_1","saq39_2"], axis=1)

test_2020["saq40"] = test_2020["saq40_1"] + test_2020["saq40_2"]
test_2020 = test_2020.drop(columns=["saq40_1","saq40_2"], axis=1)

test_2020["saq41"] = test_2020["saq41_1"] + test_2020["saq41_2"]
test_2020 = test_2020.drop(columns=["saq41_1","saq41_2"], axis=1)

test_2020["saq42"] = test_2020["saq42_1"] + test_2020["saq42_2"]
test_2020 = test_2020.drop(columns=["saq42_1","saq42_2"], axis=1)

test_2020["saq43"] = test_2020["saq43_1"] + test_2020["saq43_2"]
test_2020 = test_2020.drop(columns=["saq43_1","saq43_2"], axis=1)

test_2020["saq44"] = test_2020["saq44_1"] + test_2020["saq44_2"]
test_2020 = test_2020.drop(columns=["saq44_1","saq44_2"], axis=1)

# 처리

In [ ]:
# 문자열 공백 제거
train_2020['bq4_1a']=train_2020['bq4_1a'].str.replace(" ", "")
train_2020['bq4_1b']=train_2020['bq4_1b'].str.replace(" ", "")
train_2020['bq4_1c']=train_2020['bq4_1c'].str.replace(" ", "")
train_2020['bq5_2']=train_2020['bq5_2'].str.replace(" ", "")
train_2020['bq20_1']=train_2020['bq20_1'].str.replace(" ", "")
train_2020['bq26_1']=train_2020['bq26_1'].str.replace(" ", "")

# 영어 대문자로 통일
train_2020['bq4_1a']=train_2020['bq4_1a'].str.upper()
train_2020['bq4_1b']=train_2020['bq4_1b'].str.upper()
train_2020['bq4_1c']=train_2020['bq4_1c'].str.upper()
train_2020['bq5_2']=train_2020['bq5_2'].str.upper()
train_2020['bq20_1']=train_2020['bq20_1'].str.upper()
train_2020['bq26_1']=train_2020['bq26_1'].str.upper()

In [ ]:
# 문자열 공백 제거
test_2020['bq4_1a']=test_2020['bq4_1a'].str.replace(" ", "")
test_2020['bq4_1b']=test_2020['bq4_1b'].str.replace(" ", "")
test_2020['bq4_1c']=test_2020['bq4_1c'].str.replace(" ", "")
test_2020['bq5_2']=test_2020['bq5_2'].str.replace(" ", "")
test_2020['bq20_1']=test_2020['bq20_1'].str.replace(" ", "")
test_2020['bq26_1']=test_2020['bq26_1'].str.replace(" ", "")

# 영어 대문자로 통일
test_2020['bq4_1a']=test_2020['bq4_1a'].str.upper()
test_2020['bq4_1b']=test_2020['bq4_1b'].str.upper()
test_2020['bq4_1c']=test_2020['bq4_1c'].str.upper()
test_2020['bq5_2']=test_2020['bq5_2'].str.upper()
test_2020['bq20_1']=test_2020['bq20_1'].str.upper()
test_2020['bq26_1']=test_2020['bq26_1'].str.upper()

In [ ]:
for j in range(8122):
    if train_2020.bq26_1.iloc[j] == '공':
      train_2020.bq26_1.iloc[j] = '공고공대'
for j in range(8122):
    if train_2020.bq26_1.iloc[j] == '농':
      train_2020.bq26_1.iloc[j] = '농업'
for j in range(8122):
    if train_2020.bq26_1.iloc[j] == '문':
      train_2020.bq26_1.iloc[j] = '문과'
for j in range(8122):
    if train_2020.bq26_1.iloc[j] == '외':
      train_2020.bq26_1.iloc[j] = '외국어'
for j in range(8122):
    if train_2020.bq26_1.iloc[j] == '상':
      train_2020.bq26_1.iloc[j] = '상고상업상경'
for j in range(8122):
    if train_2020.bq26_1.iloc[j] == '약':
      train_2020.bq26_1.iloc[j] = '약학약사'
for j in range(8122):
    if train_2020.bq26_1.iloc[j] == '의':
      train_2020.bq26_1.iloc[j] = '의사전문의'
for j in range(8122):
    if train_2020.bq26_1.iloc[j] == '이':
      train_2020.bq26_1.iloc[j] = '이과'

In [ ]:
for j in range(8122):
    if test_2020.bq26_1.iloc[j] == '공':
      test_2020.bq26_1.iloc[j] = '공고공대'
for j in range(8122):
    if test_2020.bq26_1.iloc[j] == '농':
      test_2020.bq26_1.iloc[j] = '농업'
for j in range(8122):
    if test_2020.bq26_1.iloc[j] == '문':
      test_2020.bq26_1.iloc[j] = '문과'
for j in range(8122):
    if test_2020.bq26_1.iloc[j] == '외':
      test_2020.bq26_1.iloc[j] = '외국어'
for j in range(8122):
    if test_2020.bq26_1.iloc[j] == '상':
      test_2020.bq26_1.iloc[j] = '상고상업상경'
for j in range(8122):
    if test_2020.bq26_1.iloc[j] == '약':
      test_2020.bq26_1.iloc[j] = '약학약사'
for j in range(8122):
    if test_2020.bq26_1.iloc[j] == '의':
      test_2020.bq26_1.iloc[j] = '의사전문의'
for j in range(8122):
    if test_2020.bq26_1.iloc[j] == '이':
      test_2020.bq26_1.iloc[j] = '이과'

In [ ]:
testmajorfix = pd.read_csv('/content/gdrive/MyDrive/테스트학과수정용4.csv')

for i in range(235):
  train_2020.bq26_1 = train_2020.bq26_1.str.replace(testmajorfix.수정전.iloc[i], testmajorfix.수정후.iloc[i])

for i in range(235):
  test_2020.bq26_1 = test_2020.bq26_1.str.replace(testmajorfix.수정전.iloc[i], testmajorfix.수정후.iloc[i])

In [ ]:
train_2020.bq26_1 = train_2020.bq26_1.str.replace("-", "")
test_2020.bq26_1 = test_2020.bq26_1.str.replace("-", "")

In [ ]:
train_2020["key_word"] = train_2020["bq4_1a"] + " " + train_2020["bq4_1b"] + " " + train_2020["bq4_1c"] + " " + train_2020["bq5_2"] + " " + train_2020["bq20_1"] + " " + train_2020["bq26_1"]

In [ ]:
test_2020["key_word"] = test_2020["bq4_1a"] + " " + test_2020["bq4_1b"] + " " + test_2020["bq4_1c"] + " " + test_2020["bq5_2"] + " " + test_2020["bq20_1"] + " " + test_2020["bq26_1"]

In [ ]:
y = train_2020.knowcode

In [ ]:
train_2020 = train_2020.drop(columns=["bq4_1a","bq4_1b","bq4_1c",'bq5_2',"bq20_1","bq26_1"], axis=1)
test_2020 = test_2020.drop(columns=["bq4_1a","bq4_1b","bq4_1c",'bq5_2',"bq20_1","bq26_1"], axis=1)

In [ ]:
keyword = pd.read_csv('/content/gdrive/MyDrive/키워드카운트수정6.csv')
keyword

,word
0,3D
1,AFP
2,AKLU
3,AP
4,ASNT
...,...
888,회복
889,회사
890,회전익
891,회화


In [ ]:
k_list = []
for i in range(893):
  k_list.append(str(keyword.iloc[i,0]))

In [ ]:
plus_col = []
for w in range(len(k_list)):
    plus_col.append([0 for i in range(len(train_2020))])
    
for idx in tqdm(range(len(train_2020))):
    temp_str = str(train_2020.key_word.iloc[idx])
    for i in range(len(k_list)):
        plus_col[i][idx] = temp_str.count(k_list[i])
        
for i in range(len(plus_col)):
    train_2020[k_list[i]] = plus_col[i]
train_2020

100%|██████████| 8122/8122 [00:03<00:00, 2324.22it/s]


,idx,vq1,vq2,vq3,vq4,vq5,vq6,vq7,vq8,vq9,vq10,vq11,vq12,vq13,bq1,bq2,bq3,bq4,bq5,bq5_1,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13_1,bq13_2,bq13_3,bq14_1,bq14_2,bq14_3,bq14_4,...,항해,항해사,해기사,해설,해양,해외,행정,헤드셋,헤어,헬기,헬리,헬멧,현미경,현장,혈압,협회,호루라기,호스피스,호텔,호흡기,홍보,화공,화물,화약,화장품,화재,화학,화학공,화훼,환경,환경공,환자,활용,회계,회계사,회복,회사,회전익,회화,훈련
0,27127,2,4,5,3,4,4,4,4,4,3,4,5,4,16,5,3,1,2,0,7,5,4,4,4,4,4,4,3,3,4,4,4,1,5,5,4,5,5,5,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,27128,4,4,3,4,4,3,4,4,4,3,4,4,4,15,2,2,1,1,4,4,5,4,4,4,4,4,4,5,5,5,4,4,2,5,4,3,4,4,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,27129,4,4,4,3,4,3,4,4,4,4,4,4,4,3,1,4,2,2,0,5,5,4,4,4,3,4,4,4,4,4,4,4,2,4,4,4,4,3,4,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,27130,4,4,4,4,4,3,4,2,3,4,4,2,4,16,5,3,1,1,2,4,5,4,3,3,4,4,3,4,3,3,3,4,3,4,4,2,2,3,4,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,27131,4,3,5,3,4,2,4,4,3,4,4,4,5,6,1,2,1,2,0,7,5,4,4,3,4,4,4,3,4,5,5,4,2,4,4,2,4,3,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8117,35244,4,3,2,4,5,2,4,4,2,4,4,3,5,13,2,5,2,2,0,3,6,3,5,5,5,4,5,5,3,4,4,3,2,4,4,5,4,4,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8118,35245,3,2,3,4,3,3,4,4,3,2,4,2,3,11,1,2,2,2,0,2,5,3,2,4,4,5,4,3,2,4,3,3,3,4,3,4,5,3,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8119,35246,4,3,3,4,4,5,4,2,3,2,3,2,2,11,1,3,1,2,0,7,5,4,3,3,4,3,4,4,3,3,4,4,2,3,3,4,3,4,4,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8120,35247,3,3,3,3,3,4,3,3,3,2,3,3,3,3,1,2,1,2,0,6,5,4,4,4,4,4,4,3,3,3,4,3,3,3,3,3,2,3,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
plus_col = []
for w in range(len(k_list)):
    plus_col.append([0 for i in range(len(test_2020))])
    
for idx in tqdm(range(len(test_2020))):
    temp_str = str(test_2020.key_word.iloc[idx])
    for i in range(len(k_list)):
        plus_col[i][idx] = temp_str.count(k_list[i])
        
for i in range(len(plus_col)):
    test_2020[k_list[i]] = plus_col[i]
test_2020

100%|██████████| 8122/8122 [00:03<00:00, 2296.54it/s]


,idx,vq1,vq2,vq3,vq4,vq5,vq6,vq7,vq8,vq9,vq10,vq11,vq12,vq13,bq1,bq2,bq3,bq4,bq5,bq5_1,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13_1,bq13_2,bq13_3,bq14_1,bq14_2,bq14_3,bq14_4,...,항해,항해사,해기사,해설,해양,해외,행정,헤드셋,헤어,헬기,헬리,헬멧,현미경,현장,혈압,협회,호루라기,호스피스,호텔,호흡기,홍보,화공,화물,화약,화장품,화재,화학,화학공,화훼,환경,환경공,환자,활용,회계,회계사,회복,회사,회전익,회화,훈련
0,27127,4,4,4,4,3,4,4,4,3,4,3,4,4,17,1,3,1,1,4,4,2,4,5,4,4,4,4,2,3,4,4,3,1,4,4,5,4,3,4,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,27128,5,3,3,4,4,3,4,3,4,4,5,4,3,17,1,1,1,2,0,6,5,5,4,4,4,4,4,5,9,9,9,4,2,4,4,4,4,4,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,27129,4,3,4,3,3,4,4,2,4,4,4,3,4,10,1,2,2,2,0,1,5,4,4,4,4,4,4,4,4,4,4,4,2,4,4,4,4,4,4,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,27130,4,4,3,4,3,2,3,3,3,4,3,3,5,13,2,3,2,2,0,2,6,4,5,3,5,4,5,4,3,5,5,4,2,5,4,2,4,4,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,27131,3,4,4,3,4,4,4,4,4,3,4,4,4,11,1,2,1,1,1,4,1,4,3,3,4,3,4,3,3,4,4,4,3,4,3,4,4,4,4,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8117,35244,2,2,4,3,3,2,3,2,3,2,2,3,3,3,1,4,1,2,0,7,4,3,2,3,3,2,3,3,2,3,3,3,4,3,3,4,4,3,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8118,35245,4,4,4,3,3,4,3,4,3,3,4,3,4,19,1,1,2,2,0,1,2,3,2,2,4,3,3,3,3,3,3,3,4,3,3,3,2,3,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8119,35246,4,3,2,3,3,2,4,2,3,4,4,4,4,4,2,4,1,2,0,3,4,3,4,4,4,4,4,3,4,4,4,4,2,4,4,4,4,4,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8120,35247,4,4,3,3,4,3,2,2,3,4,3,3,4,6,1,3,1,1,1,4,5,4,4,4,4,4,4,4,4,3,3,3,2,3,3,3,4,4,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_2020 = train_2020.drop(columns=["idx", "key_word"], axis=1)
test_2020 = test_2020.drop(columns=["idx", "key_word"], axis=1)

In [ ]:
train_2020.to_csv('/content/gdrive/MyDrive/KNOW_data/train/train_2020_표석규.csv', index=False)
test_2020.to_csv('/content/gdrive/MyDrive/KNOW_data/test/test_2020_표석규.csv', index=False)

# 카테고리컬 처리

In [ ]:
train_2020 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/train/train_2020_중간.csv')
test_2020 = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/test/test_2020_중간.csv')

In [ ]:
train_2020['bq1'] = pd.Categorical(train_2020.bq1)
train_2020['bq2'] = pd.Categorical(train_2020.bq2)
train_2020['bq21_1'] = pd.Categorical(train_2020.bq21_1)
train_2020['bq21_2'] = pd.Categorical(train_2020.bq21_2)
train_2020['bq21_3'] = pd.Categorical(train_2020.bq21_3)
train_2020['bq27_2'] = pd.Categorical(train_2020.bq27_2)

test_2020['bq1'] = pd.Categorical(test_2020.bq1)
test_2020['bq2'] = pd.Categorical(test_2020.bq2)
test_2020['bq21_1'] = pd.Categorical(test_2020.bq21_1)
test_2020['bq21_2'] = pd.Categorical(test_2020.bq21_2)
test_2020['bq21_3'] = pd.Categorical(test_2020.bq21_3)
test_2020['bq27_2'] = pd.Categorical(test_2020.bq27_2)

In [ ]:
y = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/train/train_2020.csv')
y = y.knowcode

# 모델

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

!pip install catboost
from catboost import CatBoostClassifier

     |████████████████████████████████| 76.1 MB 70 kB/s 


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_2020, y, test_size=0.1, stratify=y, random_state=777)

In [ ]:
md = RandomForestClassifier(n_estimators=500, random_state=777)
md.fit(X_train, y_train)

RandomForestClassifier(n_estimators=500, random_state=777)

In [ ]:
y_train_hat = md.predict(X_train)
print(accuracy_score(y_train,y_train_hat))
y_valid_hat = md.predict(X_valid)
print(accuracy_score(y_valid, y_valid_hat))
print(f1_score(y_train,y_train_hat, average='macro'))
print(f1_score(y_valid, y_valid_hat, average='macro'))

1.0
0.7134071340713407
1.0
0.6707812361443646


In [ ]:
md = RandomForestClassifier(n_estimators=1400, random_state=777)
md.fit(train_2020, y)

RandomForestClassifier(n_estimators=1400, random_state=777)

In [ ]:
sample = pd.read_csv('/content/gdrive/MyDrive/KNOW_data/sample_submission_수정7.csv')

In [ ]:
testpred = md.predict(test_2020)

for i in range(8122):
  sample.knowcode.iloc[i+27109] = testpred[i]

In [ ]:
sample.to_csv('/content/gdrive/MyDrive/KNOW_data/sample_submission_수정8.csv', index=False)